# Начало

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip -q install openai
!pip -q install langchain tiktoken
!pip -q install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 78.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import tiktoken
from langchain.chat_models import ChatOpenAI

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

import getpass
import openai
import os

from IPython.display import HTML
from difflib import SequenceMatcher

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Эксперименты 1

### О себе

Промт для генерации "о себе" по интересам пользователя

In [ ]:
# Должность
position = 'Data Scientist'

# О себе
about = 'Интересны нейросети, занимаюсь изучением python'

# System Content
system_content = f'''Ты hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Соискатель написал о себе, оптимизируй этот текст,
и если надо, то дополни, чтобы он максимально был релевантным должности соискателя.
Напиши лаконично от первого лица в деловом стиле, как будто это ты устраиваешься на должность {position}.
Не пиши приветствие. В ответе уложись примерно в 300 токенов'''

# User Content
user_content = about

Подсчёт токенов

In [ ]:
messages = [
    SystemMessage(content=system_content),
    HumanMessage(content=user_content)
    ]

# Количество токенов
chat = ChatOpenAI(temperature=0)
chat.get_num_tokens_from_messages(messages)

209

Ответ ChatGPT

In [ ]:
answer = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {
            'role': 'system',
            'content': system_content
        },
        {
            'role': 'user',
            'content': about
        }
    ]
)

In [ ]:
message = answer['choices'][0]['message']['content']
message

'Я заинтересован в разработке и применении нейронных сетей в качестве инструментов анализа данных и машинного обучения. В настоящее время активно изучаю язык программирования Python и его библиотеки для работы с данными, такие как TensorFlow и PyTorch. Моя цель - применять современные методы анализа данных для нахождения закономерностей и создания прогностических моделей. Я владею навыками предобработки и очистки данных, а также статистического анализа и визуализации. У меня есть опыт работы с большими объемами данных и использованием алгоритмов машинного обучения, таких как регрессия, кластеризация и классификация. Я стремлюсь к постоянному обучению и совершенствованию, чтобы быть в курсе последних тенденций в области Data Science.'

### Проверка резюме

In [ ]:
file = '/content/drive/MyDrive/UII/Terra/hh_filtered (1).csv'
df = pd.read_csv(file, low_memory=False)

In [ ]:
# Список столбцов
list(df)

['Пол, возраст',
 'ЗП',
 'Ищет работу на должность:',
 'Город',
 'Занятость',
 'График',
 'Опыт (двойное нажатие для полной версии)',
 'Последенее/нынешнее место работы',
 'Последеняя/нынешняя должность',
 'Образование и ВУЗ',
 'Обновление резюме',
 'Авто']

In [ ]:
# номер строки
num = 4
df[num:num+1]

,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
4,"Мужчина , 54 года , родился 27 июня 1964",180000 руб.,ведущий разработчик oracle,"Москва , м. Бульвар Рокоссовского , не готов ...",полная занятость,"гибкий график, полный день, удаленная работа",Опыт работы 30 лет 9 месяцев Ведущий разработ...,Государственная корпорация “Банк развития и в...,Бизнес-аналитик / Разработчик ПО,Высшее образование 1987 Московский авиационны...,10.08.2018 12:00,Не указано


In [ ]:
num = 11
position = df['Ищет работу на должность:'][num]
education = df['Образование и ВУЗ'][num]
experience = df['Опыт (двойное нажатие для полной версии)'][num]
print('Должность:', position)
print(f'Образование [{len(education)}]:', education)
print(f'Опыт работы [{len(experience)}]:', experience)

Должность: front-end web developer
Образование [164]: Высшее образование 2018 University of the People BS, Computer Science 2015 Udacity Nanodegrees, Front-end web developer 2009 КГУКИ ИБФ, Прикладная информатика в СКС
Опыт работы [962]: Опыт работы 10 лет 4 месяца  Август 2017 — по настоящее время 1 год 10 месяцев Klerk.ru www.klerk.ru Frontend-разработчик Senior frontend developer. Vue.js, Webpack.  Март 2014 — Август  2017 3 года 6 месяцев Self-employed Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Web developer Web development  Март 2010 — Март  2014 4 года 1 месяц ООО ТД "Суворовский редут - Кубань" Продукты питания ... Кондитерские изделия (продвижение, оптовая торговля) Кондитерские изделия (производство) Хлебобулочные изделия (продвижение, оптовая торговля) Хлебобулочные изделия (производство) Программист 1С Автоматизация бизнес-процессов компании.  Июль 2006 — Июль  2007 1 год 1 месяц НОУ БКЦ "Антарес" Информационные технолог

In [ ]:
system_content = f'''Ты опытный hr консультант, который помогает соискателю грамотно составить резюме,
чтобы максимально соответствовать должности {position}. Соискатель написал резюме, которое содержит образование соискателя,
его опыт работы. Проанализируй это резюме и напиши рекомендации по его корректировке, чтобы максимально соответствовать должности {position}'''

user_content = f'''
Образование: {education}
Опыт работы: {experience}'''

In [ ]:
answer = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {
            'role': 'system',
            'content': system_content
        },
        {
            'role': 'user',
            'content': user_content
        }
    ]
)

In [ ]:
message = answer['choices'][0]['message']['content']
print(message)

Резюме соискателя нуждается в некоторых корректировках, чтобы максимально соответствовать должности front-end web developer. Вот некоторые рекомендации:

1. Образование: 
- Указать только высшее образование, связанное с информационными технологиями. Остальные программы обучения и курсы можно упомянуть в разделе "Дополнительное образование" или "Профессиональное развитие".

2. Опыт работы:
- Опишите свой опыт работы в обратном хронологическом порядке с указанием дат, компаний и должностей. 
- Для каждой позиции укажите ключевые обязанности и достигнутые результаты, чтобы продемонстрировать свои навыки и качества, связанные с front-end web development.
- Укажите использованные технологии, языки программирования и инструменты.

Вот пример, как можно откорректировать резюме:

Образование:
- Высшее образование: 
   - 2018 University of the People, BS, Computer Science
 
Дополнительное образование и курсы:
- 2015 Udacity Nanodegrees, Front-end Web Developer

Опыт работы:
   
Klerk.ru - Front

Подсчёт токенов

In [ ]:
messages = [
    SystemMessage(content=system_content),
    HumanMessage(content=user_content)
    ]

# Количество токенов
chat = ChatOpenAI(temperature=0)
chat.get_num_tokens_from_messages(messages)

689

### Проверка орфографии

In [ ]:
!pip -q install pyspellchecker

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(language='ru')
spell.correction(experience)

'Опыт работы 10 лет 4 месяца  Август 2017 — по настоящее время 1 год 10 месяцев Klerk.ru www.klerk.ru Frontend-разработчик Senior frontend developer. Vue.js, Webpack.  Март 2014 — Август  2017 3 года 6 месяцев Self-employed Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Web developer Web development  Март 2010 — Март  2014 4 года 1 месяц ООО ТД "Суворовский редут - Кубань" Продукты питания ... Кондитерские изделия (продвижение, оптовая торговля) Кондитерские изделия (производство) Хлебобулочные изделия (продвижение, оптовая торговля) Хлебобулочные изделия (производство) Программист 1С Автоматизация бизнес-процессов компании.  Июль 2006 — Июль  2007 1 год 1 месяц НОУ БКЦ "Антарес" Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Программист отдела внедрения Внедрение ПО 1С Предприятие 7.7, 8'

In [ ]:
from difflib import SequenceMatcher

# highlight colors
# you may change these values according to your preferences
color_delete = '#FF8888'  # highlight color for deletions
color_insert = '#44FF44'  # highlight color for insertions
color_replace = '#EEEE88' # highlight color for replacements

# the common format string used for highlighted segments
f_str = '<span style="background: {};">{}</span>'

# given two strings (a, b), getFormattedDiff returns the HTML formatted strings (formatted_a, formatted_b)
def getFormattedDiff(a, b):
    # initialize the sequence matcher
    s = SequenceMatcher(None, a, b)

    # stringbuilders for the formatted strings
    formatted_a = []
    formatted_b = []

    # iterate through all char blocks
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if tag == 'equal':
            # if the blovks are the same, append block to both strings without any formatting
            formatted_a.append(a[i1:i2])
            formatted_b.append(b[j1:j2])
        elif tag == 'delete':
            # if this is a deletion block, append block to the first string with the delete highlight
            formatted_a.append(f_str.format(color_delete, a[i1:i2]))
        elif tag == 'insert':
            # if this is a insertion block, append block to the second string with the insert highlight
            formatted_b.append(f_str.format(color_insert, b[j1:j2]))
        elif tag == 'replace':
            # if this is a replacement block, append block to both strings with the replace highlight
            formatted_a.append(f_str.format(color_replace, a[i1:i2]))
            formatted_b.append(f_str.format(color_replace, b[j1:j2]))

    # return the formatted strings
    return ''.join(formatted_a), ''.join(formatted_b)

In [ ]:
cases = [
    (education, spell.correction(education)),
    (experience, spell.correction(experience)),
]

In [ ]:
from IPython.display import HTML, display

for a, b in cases:
    formatted_a, formatted_b = getFormattedDiff(a, b)
    display(HTML(formatted_a))
    display(HTML(formatted_b))
    print()

In [ ]:
cases = [
    ('Пример корректного текста', 'примеры некоректнго ткста'),
]

In [ ]:
for a, b in cases:
    formatted_a, formatted_b = getFormattedDiff(a, b)
    display(HTML(formatted_a))
    display(HTML(formatted_b))
    print()

# Эксперименты 2

## Резюме

In [ ]:
name = 'Порфирий Прокофьевич'
city = 'Новополис'
position = 'Data Scientist'

In [ ]:
about = '''
Сдан экзамен P1 CIMA
Средний бал: 4,6

Имея изначально склонность к аналитике, решил добавить технологические стек и уже более углублено ее развивать в направлении Data Science.

Личные качества:
- Нахожу общий язык с коллегами
- Дисциплинирован.
- Постоянно обучаюсь новому.
- Стремлюсь к профессиональному и карьерному росту.
- В работе соблюдаю сроки выполнения задач.
- Способен длительное время заниматься решением одной проблемы.
- Умею работать с большим объёмом информации.
- Умею хранить конфиденциальную информацию.
- Способен исследовать, проверять факты и собирать точные данные.
'''

In [ ]:
experience = [
    {
        'date_start': '06.2022',
        'date_end': '',
        'organisation': 'Элерон (Европейская химчистка Apetta)',
        'city': 'Санкт-Петербург',
        'web': 'apetta.ru',
        'industry': 'Услуги для населения',
        'position': 'Бизнес-аналитик',
        'experience_type': 'Основная работа',
        'activity': 'Аналитика в Power BI, Excel. Использование базы данных. Построение автоматизированных отчетов.',
    },
    {
        'date_start': '09.2021',
        'date_end': '',
        'organisation': 'Проектная деятельность',
        'city': 'Санкт-Петербург',
        'web': 'github.com/Litvinov-Anton/Portfolio',
        'industry': '',
        'position': 'Data Scientist',
        'experience_type': 'Фриланс',
        'activity': '''
Деятельность:
Коммерческий проект по созданию отчета в Power BI, для компании Data Sfera.

Личный проект. Создал проект по отнесению кондитерских изделий к классу с помощью логистической регрессии, на основании данных Kaggle.

Личный проект. Создал несколько финансовых отчетов в Power BI, DataStudio.

Личный проект. Создал проекты по предсказанию прибыли с помощью линейной регрессии, на основании данных Kaggle.

Личный проект. Составил финансовую модель с помощью Excel.

Личный проект. Создал проект по классификации с помощью метода К-Средних, на основании данных Kaggle.
        ''',
    },
    {
        'date_start': '10.2021',
        'date_end': '11.2021',
        'organisation': 'ООО "УК "Клин-Тим" (промышленный клининг)',
        'city': 'Санкт-Петербург',
        'web': 'cleanteam.ru',
        'industry': '',
        'position': 'Экономист',
        'experience_type': 'Основная работа',
        'activity': '''
- Ведение управленческого учета компании (2 ЦФО)
- Сбор данных и формирование отчетов по маржинальности объектов (1С УПП, Excel)
- План-факторный анализ полученных данных, построение отчетности и визуализация данных (Power BI)
- Ввод данных смет в программу учета
- Подготовка аналитических справок, статистик по итогам работы за период.
- Ведение электронного архива смет

Ключевые результат:
внедрил инструмента PowerBI визуализации в процесс предоставления отчётности
''',
    },
    {
        'date_start': '06.2020',
        'date_end': '05.2021',
        'organisation': '"Терра -Авто" КЦ" (ГК «Прагматика») - один из крупнейших Российских автодилеров',
        'city': 'Санкт-Петербург',
        'web': 'www.terravto.ru',
        'industry': 'Автомобильный бизнес',
        'position': 'Экономист',
        'experience_type': 'Основная работа',
        'activity': '''
-Составление бюджета компании (2 ЦФО): планирование показателей операционного производства, выручки от продаж, ФОТа, (OPEX, CAPEX)
-План-факторный анализ полученных данных
-Составление управленческой отчетности (Excel, SQL, 1С)
-Составление экономической и финансовой отчётности для внешних контрагентов по запросу
- Контроль исполнения платежного календаря
Ключевые результаты:
Разработал и внедрил систему для расчета мотивации персонала по компании за счет введения углубленной аналитики по продажам.
''',
    },
]

In [ ]:
education = [
    {
        'date_start': '2017',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'СПбГЭУ',
        'faculty': 'Экономики, Мировая экономика',
    },
    {
        'date_start': '2015',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'Санкт-Петербургский государственный инженерно-экономический университет "ИНЖЭКОН", Санкт-Петербург',
        'faculty': 'Экономики, Национальная экономика',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Техническая Академия РосАтома',
        'faculty': 'Деловой английский',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Экономика и финансы',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Разработка и реализация маркетинговой стратегии',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Бизнес-модели компании и операционная эффективность',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Университет Синергия',
        'faculty': 'Data Scientist',
    },
    {
        'date_start': '2021',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'НИУ ИТМО, Санкт-Петербург',
        'faculty': 'Технологии анализа данных',
    },
]

## Функции обработки данных

**Работа с ChatGPT**

In [ ]:
# подсчёт токенов
def count_tokens(system_content, user_content, temperature=0):
    messages = [
        SystemMessage(content=system_content),
        HumanMessage(content=user_content)
        ]

    # Количество токенов
    chat = ChatOpenAI(temperature=temperature)
    return chat.get_num_tokens_from_messages(messages)


# генерация с помощью chatGPT
def gpt(system_content, user_content, temperature=0):

    messages = [
        {'role': 'system', 'content': system_content},
        {'role': 'user', 'content': user_content }]

    answer = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        temperature=0)

    message = answer['choices'][0]['message']['content']
    prompt_tokens = answer['usage']['prompt_tokens']
    completion_tokens = answer['usage']['completion_tokens']
    total_tokens = answer['usage']['total_tokens']

    return message, prompt_tokens, completion_tokens, total_tokens


def gpt_print(system_content, user_content, temperature=0, show_message=True):
    # вывод сгенерированных данных с помощью chatGPT
    tokens = count_tokens(system_content, user_content)
    result = gpt(system_content, user_content)
    message, prompt_tokens, completion_tokens, total_tokens = result

    if show_message:
        print(message)
        print('-' * 21)

    print('Фактическое количество токенов')
    print('прогноз:', tokens)
    print('промт:', prompt_tokens)
    print('ответ:', completion_tokens)
    print('всего:', total_tokens)

    return message

**Сравнение текстов**

In [ ]:
from difflib import SequenceMatcher

# highlight colors
# you may change these values according to your preferences
color_delete = '#FF8888'  # highlight color for deletions
color_insert = '#44FF44'  # highlight color for insertions
color_replace = '#EEEE88' # highlight color for replacements

# the common format string used for highlighted segments
f_str = '<span style="background: {};">{}</span>'

# given two strings (a, b), getFormattedDiff returns the HTML formatted strings (formatted_a, formatted_b)
def getFormattedDiff(a, b):
    # initialize the sequence matcher
    s = SequenceMatcher(None, a, b)

    # stringbuilders for the formatted strings
    formatted_a = []
    formatted_b = []

    # iterate through all char blocks
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if tag == 'equal':
            # if the blovks are the same, append block to both strings without any formatting
            formatted_a.append(a[i1:i2])
            formatted_b.append(b[j1:j2])
        elif tag == 'delete':
            # if this is a deletion block, append block to the first string with the delete highlight
            formatted_a.append(f_str.format(color_delete, a[i1:i2]))
        elif tag == 'insert':
            # if this is a insertion block, append block to the second string with the insert highlight
            formatted_b.append(f_str.format(color_insert, b[j1:j2]))
        elif tag == 'replace':
            # if this is a replacement block, append block to both strings with the replace highlight
            formatted_a.append(f_str.format(color_replace, a[i1:i2]))
            formatted_b.append(f_str.format(color_replace, b[j1:j2]))

    # return the formatted strings
    return ''.join(formatted_a), ''.join(formatted_b)

**Обработка резюме**

In [ ]:
# Перевод данных об образовании в pandas и последующая сортировка данных
def get_education_pd(education):
    # Перевод данных в pandas
    df = pd.DataFrame(education)

    # Перевод строки в дату для дальнейшей сортировки
    df.loc[df['date_start'].str.contains('\.'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%m.%Y', errors='coerce')
    df.loc[df['date_start'].str.match(r'\d{4}$'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%Y', errors='coerce')

    # Сортировка по типу учебного заведения и году поступления
    order = ['Высшее образование', 'Дополнительное образование', 'Курсы']
    df['education_type'] = pd.Categorical(df['education_type'], ordered=True, categories=order)
    df.sort_values(['education_type', 'date_start_sort'], ascending=[True, False], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df


# Перевод данных об опыте работы в pandas и последующая сортировка данных
def get_experience_pd(experience):
    # Перевод данных в pandas
    df = pd.DataFrame(experience)

    # Перевод строки в дату для дальнейшей сортировки
    df.loc[df['date_start'].str.contains('\.'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%m.%Y', errors='coerce')
    df.loc[df['date_start'].str.match(r'\d{4}$'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%Y', errors='coerce')

    # Сортировка по типу работы и году поступления
    order = ['Фриланс', 'Основная работа']
    df['experience_type'] = pd.Categorical(df['experience_type'], ordered=True, categories=order)
    df.sort_values(['experience_type', 'date_start_sort'], ascending=[True, False], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df


# Преобразование данных об образовании из pandas в str
def get_education(education):
    df = get_education_pd(education)
    ret = ''
    education_type = ''
    for index, row in df.iterrows():
        row_dict = dict(row)
        if row['education_type'] != education_type:
            ret += row['education_type'] + ':\n\n'
            education_type = row['education_type']

        ret += row['institution']
        if row['date_start']:
            ret += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
        ret += '\n'

        ret += row['faculty'] + '\n\n'

    return ret


# Преобразование данных об образовании из pandas в str
def get_experience(experience):
    df = get_experience_pd(experience)
    ret = ''
    experience_type = ''
    for index, row in df.iterrows():
        row_dict = dict(row)
        if row['experience_type'] != experience_type:
            ret += row['experience_type'] + ':\n\n'
            experience_type = row['experience_type']

        ret += row['organisation']
        if row['date_start']:
            ret += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
        ret += '\n'

        ret += row['city'] + (f", {row['web']}" if row['web'] else '') + '\n'
        ret += f"{row['industry']}\n" if row['industry'] else ''
        ret += f"{row['position']}\n" if row['position'] else ''
        ret += row['activity'] + '\n\n'

    return ret


# Вывод полного резюме в форматированном виде (html)
def print_resume_html(name, city, position, about, education, experience):
    # общие данные
    display(HTML(f'<h2>{name}</h2>'))
    display(HTML(f'<h3>{position}</h3>'))
    display(HTML(f'<br><b>О себе:</b><br>{about}<br>'))

    # образование
    df = get_education_pd(education)
    education_type = ''
    for index, row in df.iterrows():
        row_dict = dict(row)
        if row['education_type'] != education_type:
            display(HTML(f"<h3>{row['education_type']}</h3><br>"))
            education_type = row['education_type']

        institution = row['institution']
        if row['date_start']:
            institution += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
        display(HTML(f"<b>{institution}</b><br>"))
        display(HTML(f"{row['faculty']}<br><br>"))

    # опыт работы
    df = get_experience_pd(experience)
    experience_type = ''
    for index, row in df.iterrows():
        row_dict = dict(row)
        if row['experience_type'] != experience_type:
            display(HTML(f"<h3>{row['experience_type']}</h3><br>"))
            experience_type = row['experience_type']

        organisation = row['organisation']
        if row['date_start']:
            organisation += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
        display(HTML(f"<b>{organisation}</b><br>"))

        city = row['city'] + (f", {row['web']}" if row['web'] else '')
        display(HTML(f"{city}<br>"))
        if row['industry']:
            display(HTML(f"{row['position']}<br>"))
        if row['position']:
            display(HTML(f"{row['industry']}<br>"))
        display(HTML(f"{row['activity']}<br><br>"))


# Получение полного резюме в формате str
def get_resume(name, city, position, about, education, experience):
    education = get_education(education)
    experience = get_experience(experience)
    resume = f'''Имя: {name}
Должность: {position}
О себе:
{about}

{education}
Опыт работы:

{experience}
'''

    return resume

### Результаты

In [ ]:
get_education_pd(education)

,date_start,date_end,education_type,institution,faculty,date_start_sort
0,2017,,Высшее образование,СПбГЭУ,"Экономики, Мировая экономика",2017-01-01
1,2015,,Высшее образование,Санкт-Петербургский государственный инженерно-...,"Экономики, Национальная экономика",2015-01-01
2,2022,,Курсы,Техническая Академия РосАтома,Деловой английский,2022-01-01
3,2022,,Курсы,БФУ им. И. Канта,Экономика и финансы,2022-01-01
4,2022,,Курсы,БФУ им. И. Канта,Разработка и реализация маркетинговой стратегии,2022-01-01
5,2022,,Курсы,БФУ им. И. Канта,Бизнес-модели компании и операционная эффектив...,2022-01-01
6,2022,,Курсы,Университет Синергия,Data Scientist,2022-01-01
7,2021,,Курсы,"НИУ ИТМО, Санкт-Петербург",Технологии анализа данных,2021-01-01


In [ ]:
get_experience_pd(experience)

,date_start,date_end,organisation,city,web,industry,position,experience_type,activity,date_start_sort
0,09.2021,,Проектная деятельность,Санкт-Петербург,github.com/Litvinov-Anton/Portfolio,,Data Scientist,Фриланс,\nДеятельность:\nКоммерческий проект по создан...,2021-09-01
1,06.2022,,Элерон (Европейская химчистка Apetta),Санкт-Петербург,apetta.ru,Услуги для населения,Бизнес-аналитик,Основная работа,"Аналитика в Power BI, Excel. Использование баз...",2022-06-01
2,10.2021,11.2021,"ООО ""УК ""Клин-Тим"" (промышленный клининг)",Санкт-Петербург,cleanteam.ru,,Экономист,Основная работа,\n- Ведение управленческого учета компании (2 ...,2021-10-01
3,06.2020,05.2021,"""Терра -Авто"" КЦ"" (ГК «Прагматика») - один из ...",Санкт-Петербург,www.terravto.ru,Автомобильный бизнес,Экономист,Основная работа,\n-Составление бюджета компании (2 ЦФО): плани...,2020-06-01


In [ ]:
print_resume_html(name, city, position, about, education, experience)

In [ ]:
print(get_resume(name, city, position, about, education, experience))

Имя: Порфирий Прокофьевич
Должность: Data Scientist
О себе:

Сдан экзамен P1 CIMA
Средний бал: 4,6

Имея изначально склонность к аналитике, решил добавить технологические стек и уже более углублено ее развивать в направлении Data Science.

Личные качества:
- Нахожу общий язык с коллегами
- Дисциплинирован.
- Постоянно обучаюсь новому.
- Стремлюсь к профессиональному и карьерному росту.
- В работе соблюдаю сроки выполнения задач.
- Способен длительное время заниматься решением одной проблемы.
- Умею работать с большим объёмом информации.
- Умею хранить конфиденциальную информацию.
- Способен исследовать, проверять факты и собирать точные данные.


Высшее образование:

СПбГЭУ (2017 - настоящее время)
Экономики, Мировая экономика

Санкт-Петербургский государственный инженерно-экономический университет "ИНЖЭКОН", Санкт-Петербург (2015 - настоящее время)
Экономики, Национальная экономика

Курсы:

Техническая Академия РосАтома (2022 - настоящее время)
Деловой английский

БФУ им. И. Канта (20

## Работа с промтами

### О себе

**"О себе" без учёта опыта работы**

In [ ]:
print('Исходный вариант "о себе":')
print(about)

Исходный вариант "о себе":

Сдан экзамен P1 CIMA
Средний бал: 4,6

Имея изначально склонность к аналитике, решил добавить технологические стек и уже более углублено ее развивать в направлении Data Science.

Личные качества:
- Нахожу общий язык с коллегами
- Дисциплинирован.
- Постоянно обучаюсь новому.
- Стремлюсь к профессиональному и карьерному росту.
- В работе соблюдаю сроки выполнения задач.
- Способен длительное время заниматься решением одной проблемы.
- Умею работать с большим объёмом информации.
- Умею хранить конфиденциальную информацию.
- Способен исследовать, проверять факты и собирать точные данные.



In [ ]:
promt = f'''Ты hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Соискатель написал о себе, оптимизируй этот текст,
и если надо, то дополни, чтобы он максимально был релевантным должности соискателя.
Напиши лаконично от первого лица в деловом стиле, как будто это ты устраиваешься на должность {position}.
Не пиши приветствие. В ответе уложись примерно в 300 токенов'''

In [ ]:
system_content = promt
user_content = about
tokens = count_tokens(system_content, user_content)

result = gpt(system_content, user_content)
message, prompt_tokens, completion_tokens, total_tokens = result

print(message)

Я успешно сдал экзамен P1 CIMA средним баллом 4,6. Моя изначальная склонность к аналитике побудила меня расширить свой технологический стек и углубить свои знания в области Data Science.

В моих личных качествах присутствуют:
- Умение находить общий язык с коллегами, что способствует эффективной коммуникации в команде.
- Дисциплинированность, которая помогает мне соблюдать сроки выполнения задач.
- Постоянное обучение новому, так как стремлюсь к профессиональному и карьерному росту.
- Способность длительное время заниматься решением одной проблемы, что позволяет мне глубоко анализировать и исследовать данные.
- Умение работать с большим объемом информации, что является неотъемлемой частью работы Data Scientist.
- Навык хранения конфиденциальной информации, что является важным аспектом работы с данными.
- Способность исследовать, проверять факты и собирать точные данные, что позволяет мне принимать обоснованные решения на основе анализа данных.


In [ ]:
print('Фактическое количество токенов')
print('прогноз:', tokens)
print('промт:', prompt_tokens)
print('ответ:', completion_tokens)
print('всего:', total_tokens)

Фактическое количество токенов
прогноз: 460
промт: 458
ответ: 398
всего: 856


**"О себе" с учётом опыта работы**

In [ ]:
system_content = f'''Ты hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Соискатель написал свой опыт работы,
напиши лаконично от первого лица в деловом стиле о себе по этому опыту работы,
как будто это ты устраиваешься на должность {position}. Не пиши приветствие.
В ответе уложись примерно в 300 токенов'''

user_content = get_education(education)

In [ ]:
gpt_print(system_content, user_content)

У меня высшее образование в области экономики, с фокусом на мировой и национальной экономике. Я также прошел ряд курсов, которые помогли мне развить свои навыки в области анализа данных и бизнес-стратегии.

В настоящее время я прохожу обучение в Технической Академии РосАтома, где изучаю деловой английский. Это помогает мне эффективно коммуницировать с международными коллегами и клиентами.

Также я прохожу обучение в БФУ им. И. Канта, где изучаю экономику и финансы, разработку и реализацию маркетинговых стратегий, а также бизнес-модели компании и операционную эффективность. Эти курсы помогают мне понять бизнес-процессы и применить аналитические методы для оптимизации операций.

Кроме того, я прохожу обучение в Университете Синергия, где изучаю Data Science. Это позволяет мне развивать навыки работы с данными, включая сбор, обработку, анализ и визуализацию данных.

В НИУ ИТМО я изучаю технологии анализа данных, что помогает мне освоить современные инструменты и методы анализа данных.

Мо

In [ ]:
system_content = f'''Ты hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Соискатель написал свой опыт работы,
напиши лаконично от первого лица в деловом стиле о себе по этому опыту работы,
как будто это ты устраиваешься на должность {position}. Не пиши приветствие.
Отрази только релевантную информацию относительно должности {position}, остальное не упоминай.
В ответе уложись примерно в 200 токенов'''

user_content = get_education(education)

In [ ]:
gpt_print(system_content, user_content)

У меня высшее образование в области экономики, с фокусом на мировой и национальной экономике. Кроме того, я прошел ряд курсов, связанных с различными аспектами бизнеса, включая разработку маркетинговых стратегий, бизнес-моделирование и операционную эффективность.

Однако, моя настоящая страсть - анализ данных. Я прошел обучение в университете Синергия по специальности Data Scientist, а также изучал технологии анализа данных в НИУ ИТМО. Эти курсы позволили мне освоить различные методы и инструменты для работы с данными, включая машинное обучение, статистический анализ и визуализацию данных.

Я уверен, что моя комбинация знаний в области экономики и навыков в анализе данных позволит мне эффективно решать сложные задачи Data Scientist. Я готов применить свои знания и опыт для разработки инновационных решений, оптимизации бизнес-процессов и принятия обоснованных решений на основе данных.
---------------------
Фактическое количество токенов
прогноз: 550
промт: 548
ответ: 390
всего: 938


In [ ]:
system_content = f'''Ты hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Соискатель написал свой опыт работы,
напиши лаконично от первого лица в деловом стиле о себе по этому опыту работы,
как будто это ты устраиваешься на должность {position}. Не пиши приветствие.
Пиши эмоционально нейтрально без использования слова страсть.
Отрази только релевантную информацию относительно должности {position}, остальное не упоминай.
В ответе уложись примерно в 200 токенов'''

user_content = get_education(education)

In [ ]:
gpt_print(system_content, user_content)

У меня высшее образование в области экономики, с фокусом на мировой и национальной экономике. Я также прошел ряд курсов, которые помогли мне развить навыки, необходимые для работы в области Data Science.

Я прошел обучение в Технической Академии РосАтома, где изучал деловой английский. Это помогло мне улучшить коммуникационные навыки и быть готовым работать в международной среде.

Также я прошел курсы в БФУ им. И. Канта, где изучал экономику и финансы, разработку и реализацию маркетинговой стратегии, а также бизнес-модели компании и операционную эффективность. Эти курсы помогли мне понять, как применять аналитические и стратегические подходы в бизнесе.

Я также прошел обучение в Университете Синергия по специальности Data Scientist, где изучал основы анализа данных и технологии, используемые в этой области. Это позволило мне овладеть навыками работы с большими объемами данных и использовать различные алгоритмы и модели для извлечения ценной информации.

В настоящее время я продолжаю об

### Общий анализ резюме

In [ ]:
system_content = f'''Ты опытный hr консультант, который помогает соискателю грамотно составить резюме,
чтобы максимально соответствовать должности {position}. Соискатель написал резюме, которое содержит образование соискателя,
его опыт работы. Проанализируй это резюме и напиши рекомендации по его корректировке, чтобы максимально соответствовать должности {position}'''

user_content = get_resume(name, city, position, about, education, experience)
tokens = count_tokens(system_content, user_content)
print('Прогноз токенов:', tokens)

Прогноз токенов: 1766


In [ ]:
gpt_print(system_content, user_content)

Рекомендации по корректировке резюме:

1. Заголовок: Укажите свое полное имя и должность "Data Scientist" в заголовке резюме.

2. О себе: Удалите информацию о сдаче экзамена P1 CIMA и среднем балле. Вместо этого, укажите свою мотивацию и интерес к аналитике и Data Science.

3. Личные качества: Оставьте только те качества, которые наиболее важны для должности Data Scientist. Например, "аналитический склад ума", "стрессоустойчивость", "коммуникабельность" и "умение работать с большим объемом информации".

4. Образование: Укажите только высшее образование, связанное с Data Science. Удалите информацию о курсах, которые не относятся к данной области.

5. Опыт работы: Укажите только те проекты и должности, которые связаны с Data Science. Удалите информацию о работе в других областях, если они не имеют отношения к Data Science.

6. Описание опыта работы: Для каждого проекта и должности укажите конкретные задачи, которые вы выполняли, и достигнутые результаты. Укажите использованные инструмент

### Орфография

**Проверка орфографии с помощью ChatGPT**

In [ ]:
system_content = f'''Ты опытный учитель русского языка, который помогает соискателю проверить его резюме на наличие синтаксических, орфографических и стилистических ошибок.
Проверь резюме на ошибки и укажи их, если они есть'''

user_content = get_resume(name, city, position, about, education, experience)
tokens = count_tokens(system_content, user_content)
print('Прогноз токенов:', tokens)

Прогноз токенов: 1725


In [ ]:
message = gpt_print(system_content, user_content)

Имя: Порфирий Прокофьевич
Должность: Data Scientist
О себе:

Сдал экзамен P1 CIMA
Средний балл: 4,6

Имея изначальную склонность к аналитике, решил добавить технологические стеки и уже более углубленно ее развивать в направлении Data Science.

Личные качества:
- Нахожу общий язык с коллегами.
- Дисциплинирован.
- Постоянно обучаюсь новому.
- Стремлюсь к профессиональному и карьерному росту.
- В работе соблюдаю сроки выполнения задач.
- Способен длительное время заниматься решением одной проблемы.
- Умею работать с большим объемом информации.
- Умею хранить конфиденциальную информацию.
- Способен исследовать, проверять факты и собирать точные данные.


Высшее образование:

СПбГЭУ (2017 - настоящее время)
Экономика, Мировая экономика

Санкт-Петербургский государственный инженерно-экономический университет "ИНЖЭКОН", Санкт-Петербург (2015 - настоящее время)
Экономика, Национальная экономика

Курсы:

Техническая Академия РосАтома (2022 - настоящее время)
Деловой английский

БФУ им. И. Кант

In [ ]:
formatted_user, formatted_gpt = getFormattedDiff(user_content, message)

In [ ]:
# Исходное резюме с подсветкой, что было исправлено
display(HTML(formatted_user.replace('\n', '<br>')))

In [ ]:
# Резюме, исправленное с помощью ChatGPT с подсветкой мест исправления
display(HTML(formatted_gpt.replace('\n', '<br>')))

**Проверка орфографии с помощью библиотек**

In [ ]:
!pip -q install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.0 MB/s eta 0:00:00


In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(language='ru')
result = spell.correction(user_content)

In [ ]:
formatted_user, formatted_gpt = getFormattedDiff(user_content, result)

In [ ]:
# Исходное резюме с подсветкой, что было исправлено
display(HTML(formatted_user.replace('\n', '<br>')))

### Суммаризация

**Образование**

In [ ]:
system_content = f'''Ты опытный hr в сфере data science, суммаризируй образование кандидата на должность {position} относительно сферы data science'''
user_content = get_education(education)
sum_education = gpt_print(system_content, user_content, show_message=False)

Фактическое количество токенов
прогноз: 419
промт: 417
ответ: 238
всего: 655


In [ ]:
sum_education

'Кандидат имеет высшее образование в области экономики с фокусом на мировой и национальной экономике. Он также прошел несколько курсов, связанных с различными аспектами бизнеса, включая разработку маркетинговой стратегии, бизнес-моделирование и операционную эффективность. Кроме того, он прошел курсы по технологиям анализа данных и получил сертификат Data Scientist от Университета Синергия. Это образование и курсы позволяют кандидату иметь понимание экономических аспектов и навыки в области анализа данных, что может быть полезно для работы в сфере data science.'

**Опыт работы**

In [ ]:
system_content = f'''Ты опытный hr в сфере data science, суммаризируй опыт работы кандидата на должность {position} относительно сферы data science'''
user_content = get_experience(experience)
sum_experience = gpt_print(system_content, user_content, show_message=False)

Фактическое количество токенов
прогноз: 1015
промт: 1013
ответ: 510
всего: 1523


In [ ]:
sum_experience

'Кандидат имеет опыт работы в сфере data science, который включает выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Он также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии на основе данных Kaggle, создание финансовых отчетов в Power BI и DataStudio, предсказание прибыли с использованием линейной регрессии на основе данных Kaggle и создание проекта по классификации с использованием метода К-Средних на основе данных Kaggle. \n\nКроме того, кандидат имеет опыт работы в качестве бизнес-аналитика в компании, предоставляющей услуги для населения, где он занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Он также имел опыт работы в качестве экономиста в компании, занимающейся промышленным клинингом, где он занимался управленческим учетом, сбором и анализом данных, формированием отчетов и визуализацией да

**О себе**

In [ ]:
promt = f'''Ты опытный hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Напиши о соискателе для его резюме
опираясь на образование и опыт работы соискателя.
Напиши лаконично от первого лица в деловом стиле, как будто это ты устраиваешься на должность {position}.
Не пиши приветствие'''

In [ ]:
system_content = promt
user_content = sum_education + '\n' + sum_experience
about_gpt = gpt_print(system_content, user_content, show_message=False)

Фактическое количество токенов
прогноз: 902
промт: 900
ответ: 572
всего: 1472


In [ ]:
print(about_gpt)

Мой опыт работы и образование позволяют мне успешно применять аналитические навыки в сфере data science. Я имею высшее образование в области экономики с фокусом на мировой и национальной экономике, а также сертификат Data Scientist от Университета Синергия. Кроме того, я прошел несколько курсов, связанных с различными аспектами бизнеса, включая разработку маркетинговой стратегии и операционную эффективность.

Мой опыт работы включает выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Я также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии и предсказание прибыли с использованием линейной регрессии на основе данных Kaggle. Кроме того, я имел опыт работы в качестве бизнес-аналитика, где я занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов.

Мои ключевые результаты работы включают внедрение инструмента Po

###Сопроводительное письмо

In [ ]:
promt = f'''Ты опытный hr консультант, который помогает соискателю грамотно, коротко и ясно написать сопроводительное письмо.
Напиши сопроводительное письмо на должность "{position}" от лица соискателя на основании его резюме.
Выдели навыки, опыт и личные качества соискателя, связанные с работой, которые делают его подходящим для этой должности'''

In [ ]:
system_content = promt
user_content = get_resume(name, city, position, about, education, experience)
cover_letter = gpt_print(system_content, user_content, show_message=False)

Фактическое количество токенов
прогноз: 1772
промт: 1770
ответ: 734
всего: 2504


In [ ]:
print(cover_letter)

Уважаемый рекрутер,

Я хочу выразить свой интерес к вакансии Data Scientist в вашей компании. У меня есть опыт и навыки, которые делают меня подходящим кандидатом для этой должности.

Я обладаю сильными аналитическими навыками и имею опыт работы с большим объемом данных. Мой опыт в проектной деятельности и фрилансе позволил мне развить навыки работы с различными инструментами и технологиями, такими как Power BI, Excel, SQL и Python. Я также прошел курсы по анализу данных и получил сертификат Data Scientist от Университета Синергия.

Я дисциплинирован и всегда соблюдаю сроки выполнения задач. Я постоянно обучаюсь новым навыкам и стремлюсь к профессиональному и карьерному росту. Я способен длительное время заниматься решением одной проблемы и умею работать с большим объемом информации. Кроме того, я умею хранить конфиденциальную информацию и способен исследовать, проверять факты и собирать точные данные.

Я имею высшее образование в области экономики и прохожу обучение в СПбГЭУ и Санкт-П

In [ ]:
promt = f'''Ты опытный hr консультант, который помогает соискателю грамотно, коротко и ясно написать сопроводительное письмо.
Напиши сопроводительное письмо на должность "{position}" от лица соискателя на основании его резюме.
Выдели навыки, опыт и личные качества соискателя, связанные с работой, которые делают его подходящим для этой должности.
Напиши эмоционально нейтрально в деловом стиле. Не используй обращение к рекрутеру.
Не используй прилагательные при составлении сопроводительного письма.'''

In [ ]:
system_content = promt
user_content = get_resume(name, city, position, about, education, experience)
cover_letter = gpt_print(system_content, user_content, show_message=False)

Фактическое количество токенов
прогноз: 1836
промт: 1834
ответ: 815
всего: 2649


In [ ]:
print(cover_letter)

Уважаемый рекрутер,

Я хочу подать заявку на должность Data Scientist в вашей компании. У меня есть опыт работы в аналитике и я стремлюсь развиваться в области Data Science. Я уверен, что мои навыки и опыт делают меня подходящим кандидатом для этой должности.

Я обладаю следующими личными качествами, которые помогут мне успешно выполнять свои обязанности в качестве Data Scientist:

- Я легко нахожу общий язык с коллегами, что позволяет мне эффективно работать в команде.
- Я дисциплинирован и всегда соблюдаю сроки выполнения задач.
- Я постоянно обучаюсь новому и стремлюсь к профессиональному и карьерному росту.
- Я способен длительное время заниматься решением одной проблемы, что позволяет мне глубоко анализировать данные.
- Я умею работать с большим объемом информации и хранить конфиденциальную информацию.
- Я способен исследовать, проверять факты и собирать точные данные, что является важным навыком для Data Scientist.

Я также имею образование и опыт работы, которые подтверждают мою

# Эксперименты 3

## Исходное резюме

In [ ]:
name = 'Иван Петрович Сидоров'
city = 'Новополис'
position = 'Data Scientist'

contacts = [
    {'email': 'my@email.com'},
    {'Телефон': '+7(999)999-99-99'},
]

about = '''
Сдан экзамен P1 CIMA
Средний бал: 4,6

Имея изначально склонность к аналитике, решил добавить технологические стек и уже более углублено ее развивать в направлении Data Science.

Личные качества:
- Нахожу общий язык с коллегами
- Дисциплинирован.
- Постоянно обучаюсь новому.
- Стремлюсь к профессиональному и карьерному росту.
- В работе соблюдаю сроки выполнения задач.
- Способен длительное время заниматься решением одной проблемы.
- Умею работать с большим объёмом информации.
- Умею хранить конфиденциальную информацию.
- Способен исследовать, проверять факты и собирать точные данные.
'''

skills = [
    'Python',
    'Numpy',
    'Matplotlib',
    'Excel',
    'Power BI',
    '1C',
    ]

experience = [
    {
        'date_start': '06.2022',
        'date_end': '',
        'organisation': 'Элерон (Европейская химчистка Apetta)',
        'city': 'Санкт-Петербург',
        'web': 'apetta.ru',
        'industry': 'Услуги для населения',
        'position': 'Бизнес-аналитик',
        'experience_type': 'Основная работа',
        'activity': 'Аналитика в Power BI, Excel. Использование базы данных. Построение автоматизированных отчетов.',
    },
    {
        'date_start': '09.2021',
        'date_end': '',
        'organisation': 'Проектная деятельность',
        'city': 'Санкт-Петербург',
        'web': 'github.com/Litvinov-Anton/Portfolio',
        'industry': '',
        'position': 'Data Scientist',
        'experience_type': 'Фриланс',
        'activity': '''
Деятельность:
Коммерческий проект по созданию отчета в Power BI, для компании Data Sfera.

Личный проект. Создал проект по отнесению кондитерских изделий к классу с помощью логистической регрессии, на основании данных Kaggle.

Личный проект. Создал несколько финансовых отчетов в Power BI, DataStudio.

Личный проект. Создал проекты по предсказанию прибыли с помощью линейной регрессии, на основании данных Kaggle.

Личный проект. Составил финансовую модель с помощью Excel.

Личный проект. Создал проект по классификации с помощью метода К-Средних, на основании данных Kaggle.
        ''',
    },
    {
        'date_start': '10.2021',
        'date_end': '11.2021',
        'organisation': 'ООО "УК "Клин-Тим" (промышленный клининг)',
        'city': 'Санкт-Петербург',
        'web': 'cleanteam.ru',
        'industry': '',
        'position': 'Экономист',
        'experience_type': 'Основная работа',
        'activity': '''
- Ведение управленческого учета компании (2 ЦФО)
- Сбор данных и формирование отчетов по маржинальности объектов (1С УПП, Excel)
- План-факторный анализ полученных данных, построение отчетности и визуализация данных (Power BI)
- Ввод данных смет в программу учета
- Подготовка аналитических справок, статистик по итогам работы за период.
- Ведение электронного архива смет

Ключевые результат:
внедрил инструмента PowerBI визуализации в процесс предоставления отчётности
''',
    },
    {
        'date_start': '06.2020',
        'date_end': '05.2021',
        'organisation': '"Терра -Авто" КЦ" (ГК «Прагматика») - один из крупнейших Российских автодилеров',
        'city': 'Санкт-Петербург',
        'web': 'www.terravto.ru',
        'industry': 'Автомобильный бизнес',
        'position': 'Экономист',
        'experience_type': 'Основная работа',
        'activity': '''
-Составление бюджета компании (2 ЦФО): планирование показателей операционного производства, выручки от продаж, ФОТа, (OPEX, CAPEX)
-План-факторный анализ полученных данных
-Составление управленческой отчетности (Excel, SQL, 1С)
-Составление экономической и финансовой отчётности для внешних контрагентов по запросу
- Контроль исполнения платежного календаря
Ключевые результаты:
Разработал и внедрил систему для расчета мотивации персонала по компании за счет введения углубленной аналитики по продажам.
''',
    },
]

education = [
    {
        'date_start': '2017',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'СПбГЭУ',
        'faculty': 'Экономики, Мировая экономика',
    },
    {
        'date_start': '2015',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'Санкт-Петербургский государственный инженерно-экономический университет "ИНЖЭКОН", Санкт-Петербург',
        'faculty': 'Экономики, Национальная экономика',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Техническая Академия РосАтома',
        'faculty': 'Деловой английский',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Экономика и финансы',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Разработка и реализация маркетинговой стратегии',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Бизнес-модели компании и операционная эффективность',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Университет Синергия',
        'faculty': 'Data Scientist',
    },
    {
        'date_start': '2021',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'НИУ ИТМО, Санкт-Петербург',
        'faculty': 'Технологии анализа данных',
    },
]

## Класс работы с ChatGPT

In [ ]:
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"
MODEL_GPT4 = "gpt-4-0613"

class chatgpt():
    def __init__(self, name, city, position, contacts, about, skills, education, experience, model=MODEL_TURBO_0613):
        self.name = name
        self.city = city
        self.position = position
        self.contacts = contacts
        self.about = about
        self.skills = skills
        self.education = education
        self.experience = experience
        self.model = model
        self.results = {}


    # Работа с ChatGPT

    # подсчёт токенов
    def count_tokens(self, system_content, user_content, temperature=0):
        messages = [
            SystemMessage(content=system_content),
            HumanMessage(content=user_content)
            ]

        # Количество токенов
        chat = ChatOpenAI(temperature=temperature)
        return chat.get_num_tokens_from_messages(messages)


    # генерация с помощью chatGPT со статистикой по токенам
    def gpt_gen_info(self, system_content, user_content, assistant_content=None, temperature=0, key=None):

        messages = [
            {'role': 'system', 'content': system_content},
            {'role': 'user', 'content': user_content},
            ]

        if assistant_content:
            messages.append({'role': 'assistant', 'content': assistant_content})

        answer = openai.ChatCompletion.create(
            model=self.model,
            messages=messages,
            temperature=0)

        message = answer['choices'][0]['message']['content']
        prompt_tokens = answer['usage']['prompt_tokens']
        completion_tokens = answer['usage']['completion_tokens']
        total_tokens = answer['usage']['total_tokens']

        self.results['last_result'] = message

        if key:
            self.results['key'] = message

        return message, prompt_tokens, completion_tokens, total_tokens


    # генерация с помощью chatGPT без статистикои по токенам
    def gpt_gen(self, system_content, user_content, assistant_content=None, temperature=0, key=None):
        message, _, _, _ = self.gpt_gen_info(system_content, user_content, temperature, key)
        return message


    # вывод сгенерированных данных и статистики по токенам с помощью chatGPT
    def gpt_gen_print(self, system_content, user_content, assistant_content=None, temperature=0, key=None, quiet=False):

        tokens = self.count_tokens(system_content, user_content)
        result = self.gpt_gen_info(system_content, user_content, key)
        message, prompt_tokens, completion_tokens, total_tokens = result

        print(message)
        if not quiet:
            print('-' * 21)
            print('Фактическое количество токенов')
            print('прогноз:', tokens)
            print('промт:', prompt_tokens)
            print('ответ:', completion_tokens)
            print('всего:', total_tokens)

        # return message


    # Сравнение текстов

    # highlight colors
    # you may change these values according to your preferences
    color_delete = '#FF8888'  # highlight color for deletions
    color_insert = '#44FF44'  # highlight color for insertions
    color_replace = '#EEEE88' # highlight color for replacements

    # the common format string used for highlighted segments
    f_str = '<span style="background: {};">{}</span>'

    # given two strings (a, b), getFormattedDiff returns the HTML formatted strings (formatted_a, formatted_b)
    def getFormattedDiff(self, a, b):
        # initialize the sequence matcher
        s = SequenceMatcher(None, a, b)

        # stringbuilders for the formatted strings
        formatted_a = []
        formatted_b = []

        # iterate through all char blocks
        for tag, i1, i2, j1, j2 in s.get_opcodes():
            if tag == 'equal':
                # if the blovks are the same, append block to both strings without any formatting
                formatted_a.append(a[i1:i2])
                formatted_b.append(b[j1:j2])
            elif tag == 'delete':
                # if this is a deletion block, append block to the first string with the delete highlight
                formatted_a.append(f_str.format(color_delete, a[i1:i2]))
            elif tag == 'insert':
                # if this is a insertion block, append block to the second string with the insert highlight
                formatted_b.append(f_str.format(color_insert, b[j1:j2]))
            elif tag == 'replace':
                # if this is a replacement block, append block to both strings with the replace highlight
                formatted_a.append(f_str.format(color_replace, a[i1:i2]))
                formatted_b.append(f_str.format(color_replace, b[j1:j2]))

        # return the formatted strings
        return ''.join(formatted_a), ''.join(formatted_b)


    # Обработка резюме

    # Перевод данных об образовании в pandas и последующая сортировка данных
    def get_education_pd(self, education):
        # Перевод данных в pandas
        df = pd.DataFrame(education)

        # Перевод строки в дату для дальнейшей сортировки
        df.loc[df['date_start'].str.contains('\.'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%m.%Y', errors='coerce')
        df.loc[df['date_start'].str.match(r'\d{4}$'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%Y', errors='coerce')

        # Сортировка по типу учебного заведения и году поступления
        order = ['Высшее образование', 'Дополнительное образование', 'Курсы']
        df['education_type'] = pd.Categorical(df['education_type'], ordered=True, categories=order)
        df.sort_values(['education_type', 'date_start_sort'], ascending=[True, False], inplace=True)
        df.reset_index(drop=True, inplace=True)

        return df


    # Перевод данных об опыте работы в pandas и последующая сортировка данных
    def get_experience_pd(self, experience):
        # Перевод данных в pandas
        df = pd.DataFrame(experience)

        # Перевод строки в дату для дальнейшей сортировки
        df.loc[df['date_start'].str.contains('\.'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%m.%Y', errors='coerce')
        df.loc[df['date_start'].str.match(r'\d{4}$'), 'date_start_sort'] = pd.to_datetime(df['date_start'], format='%Y', errors='coerce')

        # Сортировка по типу работы и году поступления
        order = ['Фриланс', 'Основная работа']
        df['experience_type'] = pd.Categorical(df['experience_type'], ordered=True, categories=order)
        df.sort_values(['experience_type', 'date_start_sort'], ascending=[True, False], inplace=True)
        df.reset_index(drop=True, inplace=True)

        return df


    # Преобразование данных об образовании из pandas в str
    def get_education(self, education):
        df = self.get_education_pd(education)
        ret = ''
        education_type = ''
        for index, row in df.iterrows():
            row_dict = dict(row)
            if row['education_type'] != education_type:
                ret += row['education_type'] + ':\n\n'
                education_type = row['education_type']

            ret += row['institution']
            if row['date_start']:
                ret += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
            ret += '\n'

            ret += row['faculty'] + '\n\n'

        return ret


    # Преобразование данных об образовании из pandas в str
    def get_experience(self, experience):
        df = self.get_experience_pd(experience)
        ret = ''
        experience_type = ''
        for index, row in df.iterrows():
            row_dict = dict(row)
            if row['experience_type'] != experience_type:
                ret += row['experience_type'] + ':\n\n'
                experience_type = row['experience_type']

            ret += row['organisation']
            if row['date_start']:
                ret += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
            ret += '\n'

            ret += row['city'] + (f", {row['web']}" if row['web'] else '') + '\n'
            ret += f"{row['industry']}\n" if row['industry'] else ''
            ret += f"{row['position']}\n" if row['position'] else ''
            ret += row['activity'] + '\n\n'

        return ret


    # Получение контактов
    def get_contacts(self, contacts):
        ret = ''
        for item in contacts:
            for key, value in item.items():
                ret += f'{key}: {value}\n'
        if ret:
            ret = f'Контакты:\n{ret}'

        return ret


    # Вывод полного резюме в форматированном виде (html)
    def print_resume_html(self, name, city, position, about, skills, education, experience):
        # общие данные
        display(HTML(f'<h2>{name}</h2>'))
        display(HTML(f'<h3>{position}</h3>'))
        display(HTML(f'<br><b>О себе:</b><br>{about}<br>'))
        display(HTML(f'<br><b>Ключевые навыки:</b><br>{", ".join(skills)}<br>'))

        # образование
        df = self.get_education_pd(education)
        education_type = ''
        for index, row in df.iterrows():
            row_dict = dict(row)
            if row['education_type'] != education_type:
                display(HTML(f"<h3>{row['education_type']}</h3><br>"))
                education_type = row['education_type']

            institution = row['institution']
            if row['date_start']:
                institution += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
            display(HTML(f"<b>{institution}</b><br>"))
            display(HTML(f"{row['faculty']}<br><br>"))

        # опыт работы
        df = self.get_experience_pd(experience)
        experience_type = ''
        for index, row in df.iterrows():
            row_dict = dict(row)
            if row['experience_type'] != experience_type:
                display(HTML(f"<h3>{row['experience_type']}</h3><br>"))
                experience_type = row['experience_type']

            organisation = row['organisation']
            if row['date_start']:
                organisation += ' (' + row['date_start'] + ' - ' + (row['date_end'] if row['date_end'] else 'настоящее время') + ')'
            display(HTML(f"<b>{organisation}</b><br>"))

            city = row['city'] + (f", {row['web']}" if row['web'] else '')
            display(HTML(f"{city}<br>"))
            if row['industry']:
                display(HTML(f"{row['position']}<br>"))
            if row['position']:
                display(HTML(f"{row['industry']}<br>"))
            display(HTML(f"{row['activity']}<br><br>"))


    # Получение полного резюме в формате str
    def get_resume(self, **kwargs):
        name = self.name if 'name' not in kwargs else kwargs['name']
        position = self.position if 'position' not in kwargs else kwargs['position']

        about = self.about if 'about' not in kwargs else kwargs['about']
        if about:
            about = f'''
О себе:
{about}
'''

        skills = self.skills if 'skills' not in kwargs else kwargs['skills']
        skills = ', '.join(skills)
        if skills:
            skills = f'''
Ключевые навыки:
{skills}
'''

        contacts = self.get_contacts(self.contacts if 'contacts' not in kwargs else kwargs['contacts'])
        education = self.get_education(self.education if 'education' not in kwargs else kwargs['education'])
        experience = self.get_experience(self.experience if 'experience' not in kwargs else kwargs['experience'])

        resume = f'''Имя: {name}
Должность: {position}

{contacts}
{about}
{skills}
{education}
Опыт работы:

{experience}
'''

        return resume

## Работа с промтами

In [ ]:
gpt = chatgpt(name, city, position, contacts, about, skills, education, experience)

### О себе + суммаризация

Что было замечено во время экспериментов, и как было исправлено промтами:
- восторженный стиль написания (например: "моя страсть - изучение нейронных сетей"), промт: пиши в деловом стиле
- приветствие в начале текста, промт: не пиши приветствие
- длинное размазанное описание, промт: пиши кратко и лаконично
- в некоторых промтах надо ограничивать количество слов

Прочие выводы:
- если "о себе" хорошо составлено, то генерация в целом его не улучшает, а в некоторых случаях делает хуже по восприятию

In [ ]:
# О себе, исходные данные

print(about)


Сдан экзамен P1 CIMA
Средний бал: 4,6

Имея изначально склонность к аналитике, решил добавить технологические стек и уже более углублено ее развивать в направлении Data Science.

Личные качества:
- Нахожу общий язык с коллегами
- Дисциплинирован.
- Постоянно обучаюсь новому.
- Стремлюсь к профессиональному и карьерному росту.
- В работе соблюдаю сроки выполнения задач.
- Способен длительное время заниматься решением одной проблемы.
- Умею работать с большим объёмом информации.
- Умею хранить конфиденциальную информацию.
- Способен исследовать, проверять факты и собирать точные данные.



In [ ]:
promt = f'''Ты hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Соискатель написал о себе, оптимизируй этот текст,
и если надо, то дополни, чтобы он максимально был релевантным должности соискателя.
Напиши лаконично от первого лица в деловом стиле, как будто это ты устраиваешься на должность {position}.
Не пиши приветствие'''
system_content = promt
user_content = about
sum_education = gpt.gpt_gen_print(system_content, user_content)

Я успешно сдал экзамен P1 CIMA средним баллом 4,6. Моя изначальная склонность к аналитике побудила меня добавить технологические навыки и углубить свои знания в области Data Science.

У меня есть ряд личных качеств, которые помогут мне быть эффективным Data Scientist:
- Я легко нахожу общий язык с коллегами, что способствует эффективной коммуникации и сотрудничеству.
- Я дисциплинирован и всегда соблюдаю сроки выполнения задач.
- Я постоянно обучаюсь новым навыкам и стремлюсь к профессиональному и карьерному росту.
- Я способен длительное время заниматься решением одной проблемы, что позволяет мне глубоко анализировать данные и находить оптимальные решения.
- Я умею работать с большим объемом информации и эффективно ее обрабатывать.
- Я обладаю навыками хранения конфиденциальной информации, что является важным аспектом работы с данными.
- Я способен проводить исследования, проверять факты и собирать точные данные, что помогает мне принимать обоснованные решения.

Я готов применить свои

In [ ]:
sum_education = gpt.gpt_gen_print(system_content, user_content, temperature=0.3)

Я успешно сдал экзамен P1 CIMA средним баллом 4,6. Моя изначальная склонность к аналитике побудила меня добавить технологические навыки и углубить свои знания в области Data Science.

В моих личных качествах:
- Я легко нахожу общий язык с коллегами, что помогает мне эффективно работать в команде.
- Я дисциплинирован и всегда соблюдаю сроки выполнения задач.
- Я постоянно обучаюсь новому и стремлюсь к профессиональному и карьерному росту.
- Я способен длительное время заниматься решением одной проблемы, что позволяет мне глубоко анализировать данные.
- Я умею работать с большим объемом информации и хранить конфиденциальную информацию.
- Я способен исследовать, проверять факты и собирать точные данные, что является важным навыком для Data Scientist.

Моя цель - применить свои аналитические и технические навыки в области Data Science, чтобы эффективно анализировать данные, находить новые паттерны и предоставлять ценные инсайты для бизнеса. Я готов внести свой вклад в команду Data Scientis

In [ ]:
# Суммаризация образования

system_content = f'''Ты опытный hr в сфере data science, суммаризируй образование кандидата на должность {position} относительно сферы data science'''
user_content = gpt.get_education(education)
sum_education = gpt.gpt_gen_print(system_content, user_content)

Кандидат имеет высшее образование в области экономики с фокусом на мировой и национальной экономике. Он также прошел несколько курсов, связанных с различными аспектами бизнеса, включая разработку маркетинговой стратегии, бизнес-моделирование и операционную эффективность. Кроме того, он прошел курсы по технологиям анализа данных и получил сертификат Data Scientist от Университета Синергия. Это образование и курсы позволяют кандидату иметь понимание экономических аспектов и навыки в области анализа данных, что может быть полезно для работы в сфере data science.
---------------------
Фактическое количество токенов
прогноз: 419
промт: 417
ответ: 238
всего: 655


In [ ]:
# Суммаризация опыта работы

system_content = f'''Ты опытный hr в сфере data science, суммаризируй опыт работы кандидата на должность {position} относительно сферы data science'''
user_content = gpt.get_experience(experience)
sum_experience = gpt.gpt_gen_print(system_content, user_content)

Кандидат имеет опыт работы в сфере data science, который включает выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Он также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии на основе данных Kaggle, создание финансовых отчетов в Power BI и DataStudio, предсказание прибыли с использованием линейной регрессии на основе данных Kaggle и создание проекта по классификации с использованием метода К-Средних на основе данных Kaggle. 

Кроме того, кандидат имеет опыт работы в качестве бизнес-аналитика в компании, предоставляющей услуги для населения, где он занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Он также имел опыт работы в качестве экономиста в компании, занимающейся промышленным клинингом, где он занимался управленческим учетом, сбором и анализом данных, формированием отчетов и визуализацией данны

In [ ]:
# Генерация "о себе" с учётом суммаризации образования и опыта работы

system_content = f'''Ты опытный hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Напиши о соискателе для его резюме
опираясь на образование и опыт работы соискателя.
Напиши кратко и лаконично от первого лица в деловом стиле, как будто это ты устраиваешься на должность {position}.
Не пиши приветствие'''

user_content = f'Суммаризация образования соискателя:\n{sum_education}\nСуммаризация опыта работы соискателя:\n{sum_experience}'

gpt.gpt_gen_print(system_content, user_content, quiet=True)

Я имею высшее образование в области экономики с фокусом на мировой и национальной экономике. Кроме того, я прошел несколько курсов, связанных с различными аспектами бизнеса, включая разработку маркетинговой стратегии и операционную эффективность. Я также получил сертификат Data Scientist от Университета Синергия, что позволяет мне иметь понимание экономических аспектов и навыки в области анализа данных.

У меня есть опыт работы в сфере data science, включая выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Я также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии и предсказание прибыли с использованием линейной регрессии на основе данных Kaggle.

Кроме того, у меня есть опыт работы в качестве бизнес-аналитика, где я занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Я также имел опыт работы в качестве

'Я имею высшее образование в области экономики с фокусом на мировой и национальной экономике. Кроме того, я прошел несколько курсов, связанных с различными аспектами бизнеса, включая разработку маркетинговой стратегии и операционную эффективность. Я также получил сертификат Data Scientist от Университета Синергия, что позволяет мне иметь понимание экономических аспектов и навыки в области анализа данных.\n\nУ меня есть опыт работы в сфере data science, включая выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Я также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии и предсказание прибыли с использованием линейной регрессии на основе данных Kaggle.\n\nКроме того, у меня есть опыт работы в качестве бизнес-аналитика, где я занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Я также имел опыт работы в кач

In [ ]:
print(gpt.results['last_result'])

Я имею высшее образование в области экономики с фокусом на мировой и национальной экономике. Кроме того, я прошел несколько курсов, связанных с различными аспектами бизнеса, включая разработку маркетинговой стратегии и операционную эффективность. Я также получил сертификат Data Scientist от Университета Синергия, что позволяет мне иметь понимание экономических аспектов и навыки в области анализа данных.

У меня есть опыт работы в сфере data science, включая выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Я также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии и предсказание прибыли с использованием линейной регрессии на основе данных Kaggle.

Кроме того, у меня есть опыт работы в качестве бизнес-аналитика, где я занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Я также имел опыт работы в качестве

In [ ]:
# Правка "о себе" с учётом суммаризации образования и опыта работы

system_content = f'''Ты опытный hr консультант, который помогает соискателю грамотно написать о себе,
чтобы максимально соответствовать должности {position}. Напиши о соискателе для его резюме
опираясь на образование и опыт работы соискателя.
Напиши кратко и лаконично от первого лица в деловом стиле, как будто это ты устраиваешься на должность {position}.
Не пиши приветствие. Уложись в 100 слов'''

user_content = f'Соискатель написал о себе:\n{about}\nСуммаризация образования соискателя:\n{sum_education}\nСуммаризация опыта работы соискателя:\n{sum_experience}'

gpt.gpt_gen_print(system_content, user_content, quiet=True)

У меня есть опыт работы в сфере Data Science, включая выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Я также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии на основе данных Kaggle, создание финансовых отчетов в Power BI и DataStudio, предсказание прибыли с использованием линейной регрессии на основе данных Kaggle и создание проекта по классификации с использованием метода К-Средних на основе данных Kaggle.

Кроме того, у меня есть опыт работы в качестве бизнес-аналитика, где я занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Я также имел опыт работы в качестве экономиста, где занимался управленческим учетом, сбором и анализом данных, формированием отчетов и визуализацией данных с использованием Power BI.

Мои ключевые результаты работы включают внедрение инструмента Power BI для визуализации д

'У меня есть опыт работы в сфере Data Science, включая выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Я также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии на основе данных Kaggle, создание финансовых отчетов в Power BI и DataStudio, предсказание прибыли с использованием линейной регрессии на основе данных Kaggle и создание проекта по классификации с использованием метода К-Средних на основе данных Kaggle.\n\nКроме того, у меня есть опыт работы в качестве бизнес-аналитика, где я занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Я также имел опыт работы в качестве экономиста, где занимался управленческим учетом, сбором и анализом данных, формированием отчетов и визуализацией данных с использованием Power BI.\n\nМои ключевые результаты работы включают внедрение инструмента Power BI для визуализа

In [ ]:
print(gpt.results['last_result'])

У меня есть опыт работы в сфере Data Science, включая выполнение коммерческого проекта по созданию отчета в Power BI для компании Data Sfera. Я также выполнил несколько личных проектов, включая создание модели отнесения кондитерских изделий к классу с использованием логистической регрессии на основе данных Kaggle, создание финансовых отчетов в Power BI и DataStudio, предсказание прибыли с использованием линейной регрессии на основе данных Kaggle и создание проекта по классификации с использованием метода К-Средних на основе данных Kaggle.

Кроме того, у меня есть опыт работы в качестве бизнес-аналитика, где я занимался аналитикой в Power BI и Excel, использованием базы данных и построением автоматизированных отчетов. Я также имел опыт работы в качестве экономиста, где занимался управленческим учетом, сбором и анализом данных, формированием отчетов и визуализацией данных с использованием Power BI.

Мои ключевые результаты работы включают внедрение инструмента Power BI для визуализации д

### Выявление ключевых навыков

Выявленные проблемы промтов:
- пишет много общих описаний типа "аналитическое мышление", "умение планировать" и т.д., добавлено в промт "пиши только название фреймворков и технологий", так же добавлено "пиши на английском"
- в целом выявляет очень широкий спектр навыков

In [ ]:
system_content = f'''Ты hr консультант, который помогает соискателю грамотно проанализировать его резюме,
и выявить ключевые навыки, соответствующие позиции "{position}". Проверь резюме соискателя и напиши через запятую
все навыки, которые найдёшь. Навыки должны соответствовать только должности "{position}", на которую устраивается соискатель.
Если навыки не соответствуют направлению "{position}", то не пиши их'''

user_content = gpt.get_resume()

gpt.gpt_gen_print(system_content, user_content)

Python, Numpy, Matplotlib, Excel, Power BI
---------------------
Фактическое количество токенов
прогноз: 1816
промт: 1814
ответ: 12
всего: 1826


'Python, Numpy, Matplotlib, Excel, Power BI'

In [ ]:
# Добавление промта "Пиши только название технологий и фреймворков"

system_content = f'''Ты hr консультант, который помогает соискателю грамотно проанализировать его резюме,
и выявить ключевые навыки, соответствующие позиции "{position}". Проверь резюме соискателя и напиши через запятую
все навыки, которые найдёшь. Навыки должны соответствовать только должности "{position}", на которую устраивается соискатель.
Если навыки не соответствуют направлению "{position}", то не пиши их. Допиши так же через запятую ключевые навыки,
которые должны соответствовать позиции "{position}", но которые соискатель не указал. Пиши только название технологий и фреймворков'''

user_content = gpt.get_resume()

gpt.gpt_gen_print(system_content, user_content, quiet=True)

Python, Numpy, Matplotlib, Excel, Power BI


'Python, Numpy, Matplotlib, Excel, Power BI'

In [ ]:
# Исключаем из резюме ключевые навыки

system_content = f'''Ты hr консультант, который помогает соискателю грамотно проанализировать его резюме,
и выявить ключевые навыки, соответствующие позиции "{position}". Проверь резюме соискателя и напиши через запятую
все навыки, которые найдёшь. Навыки должны соответствовать только должности "{position}", на которую устраивается соискатель.
Если навыки не соответствуют направлению "{position}", то не пиши их. Допиши так же через запятую ключевые навыки,
которые должны соответствовать позиции "{position}", но которые соискатель не указал. Пиши только название технологий и фреймворков на английском'''

user_content = gpt.get_resume(skills=[])
gpt.gpt_gen_print(system_content, user_content, temperature=0.2, quiet=True)

Python, R, SQL, Power BI, Excel, Data Analysis, Data Visualization, Machine Learning, Statistical Analysis, Regression Analysis, Logistic Regression, K-Means Clustering, Financial Modeling, Business Analytics, Time Management, Problem Solving, Communication Skills, Teamwork


'Python, R, SQL, Power BI, Excel, Data Analysis, Data Visualization, Machine Learning, Statistical Analysis, Regression Analysis, Logistic Regression, K-Means Clustering, Financial Modeling, Business Analytics, Time Management, Problem Solving, Communication Skills, Teamwork'

### Анализ резюме относительно вакансии

In [ ]:
vacancy1 = '''
Задачи
- Разработка скоринговых моделей оценки контрагентов физических и юридических лиц (сегменты B2C, B2B, B2G)
- Разработка моделей оптимизации стратегий сбора задолженности (collection scoring)
- Разработка ML моделей (включая нейросети) для автоматизации и повышения эффективности операционных процессов
- Разработка ML моделей по анализу и настройке голосовых роботов
- Участие во внедрении и тестировании ML моделей, ПО и инфраструктуры для ML
- Мониторинг эффективности действующих моделей
- Анализ эффективности внутренних и внешних информационных источников

Нам важно
- Опыт разработки скоринговых/ML моделей
- Навык работы с БД
- Владение SQL и Python
- Знание подходов к оценке эффективности моделей и финансовой эффективности их внедрения
- Навыки построения аналитических отчетов и умение визуализировать данные
- Хорошую аналитическую и математическую подготовку
- Умение копаться в данных, генерировать гипотезы, находить инсайты
- Будет большим плюсом опыт менторинга других аналитиков
'''

In [ ]:
system_content = f'''Ты опытный hr консультант, который помогает соискателю грамотно составить резюме,
чтобы максимально соответствовать должности {position}. Соискатель написал резюме, которое содержит навыки, образование соискателя,
его опыт работы. Так же приложена вакансия. Проведи анализ относительно данной вакансии, напиши рекомендации по корректировке резюме,
чтобы максимально соответствовать должности {position} и максимально подходить к данном вакансии'''

resume = gpt.get_resume()
user_content = f'Резюме соискателя:\n{resume}\n\nВакансия:\n{vacancy1}'
gpt.gpt_gen_print(system_content, user_content)

Анализ вакансии и рекомендации по корректировке резюме:

1. Вакансия требует разработку скоринговых моделей оценки контрагентов, моделей оптимизации стратегий сбора задолженности, ML моделей для автоматизации операционных процессов и анализа голосовых роботов. Рекомендуется в резюме подчеркнуть опыт и навыки в разработке и применении таких моделей.

2. Вакансия требует навык работы с базами данных и владение SQL. В резюме следует упомянуть опыт работы с базами данных и навыки работы с SQL.

3. Вакансия требует владение Python. В резюме уже указаны навыки работы с Python, но рекомендуется уточнить, что уровень владения языком позволяет разрабатывать ML модели.

4. Вакансия требует знание подходов к оценке эффективности моделей и их финансовой эффективности. В резюме следует упомянуть опыт оценки эффективности моделей и знание финансовых аспектов их внедрения.

5. Вакансия требует навыки построения аналитических отчетов и визуализации данных. В резюме уже указано владение Excel и Power B

'Анализ вакансии и рекомендации по корректировке резюме:\n\n1. Вакансия требует разработку скоринговых моделей оценки контрагентов, моделей оптимизации стратегий сбора задолженности, ML моделей для автоматизации операционных процессов и анализа голосовых роботов. Рекомендуется в резюме подчеркнуть опыт и навыки в разработке и применении таких моделей.\n\n2. Вакансия требует навык работы с базами данных и владение SQL. В резюме следует упомянуть опыт работы с базами данных и навыки работы с SQL.\n\n3. Вакансия требует владение Python. В резюме уже указаны навыки работы с Python, но рекомендуется уточнить, что уровень владения языком позволяет разрабатывать ML модели.\n\n4. Вакансия требует знание подходов к оценке эффективности моделей и их финансовой эффективности. В резюме следует упомянуть опыт оценки эффективности моделей и знание финансовых аспектов их внедрения.\n\n5. Вакансия требует навыки построения аналитических отчетов и визуализации данных. В резюме уже указано владение Exce

In [ ]:
system_content = f'''Ты опытный hr консультант, который консультирует соискателя на должность {position}.
Соискатель написал резюме, так же приложена вакансия. Проведи анализ резюме соискателя относительно данной вакансии.
Напиши в процентах, насколько данный кандидат соответствует данной вакансии, а так же перечисли то, чего ему не хватате,
чтобы устроиться на данную вакансию'''

resume = gpt.get_resume()
user_content = f'Резюме соискателя:\n{resume}\n\nВакансия:\n{vacancy1}'
gpt.gpt_gen_print(system_content, user_content, quiet=True)

Анализ резюме соискателя относительно вакансии Data Scientist:

1. Опыт разработки скоринговых/ML моделей: 80%
   Соискатель указал, что у него есть опыт работы над проектами по предсказанию прибыли с помощью линейной регрессии и классификации с помощью метода К-Средних. Это свидетельствует о его опыте работы с моделями машинного обучения.

2. Навык работы с БД: 70%
   Соискатель указал, что он умеет работать с базами данных. Однако, не указал конкретные базы данных, с которыми работал. Более подробная информация о его опыте работы с БД была бы полезна.

3. Владение SQL и Python: 80%
   Соискатель указал, что владеет языком программирования Python. Однако, не указал конкретные навыки работы с SQL. Более подробная информация о его навыках работы с SQL была бы полезна.

4. Знание подходов к оценке эффективности моделей и финансовой эффективности их внедрения: 70%
   Соискатель не указал конкретные знания по оценке эффективности моделей и их внедрения. Более подробная информация о его зна

In [ ]:
vacancy2 = '''
Вы будете:

    Взаимодействовать с заинтересованными сторонами в рамках всей организации, чтобы определить осуществимость использования данных компании для формирования бизнес-решений и монетизации данных
    Извлекать и анализировать данные из баз данных компаний для оптимизации инструментов, процессов
    Производить пользовательские модели данных и алгоритмов для применения к наборам данных;
    Формулировать процессы и инструменты для мониторинга и анализа производительности модели и точности данных;
    Представлять результаты моделирования для бизнес-аудитории, включая руководителей организации;
    Реализовывать стратегию управления бизнес-данными и давать рекомендации
    Проектировать, рефакторить, разворачивать дата продукты;
    Выдвигать в организации использование подготовленных внутри дата продуктов с помощью презентаций, документации и совместной работы бизнес функций;
    Подготавливать инструменты и автоматизировать рабочие процессы;
    Превращать неструктурированные недостатки и болевые точки в конкретные бизнес и технические требования для проверки гипотез или производства дата продуктов.

Мы ожидаем:

    Образование и соответствующий опыт в области статистики, математики, информатики или смежных областях;
    Уверенное знание SQL, Python, алгоритмов машинного обучения и статистики;
    Подтвержденный опыт создания пайплайнов машинного обучения, работающих в продуктивной среде разработки (обучение, оценка, использование в системах реального времени);
    Опыт использования облачных платформ (Azure ML, Google Cloud Platform ML, AWS SageMaker, Yandex Cloud Platform) и развертывания аналитики в продуктивной среде;
    Опыт работы со стеком Azure (Data Lake, Database, Data Factory, Data Brick и т. д.) и/или Yandex Cloud (Data Proc, Data Sphere и т. д.) – будет плюсом;
    Опыт визуализации данных с использованием Power BI/ggplot/plotly/ – будет плюсом;
    Высокие навыки коммуникации, решения проблем, стремление к обучению;
    Самостоятельного специалиста, с высокой ответственностью, готового привносить структуру в неструктурированные ситуации;
    Средний или выше среднего уровень знания английского языка (разговорный и письменный).

'''

In [ ]:
system_content = f'''Ты опытный hr консультант, который помогает соискателю грамотно составить резюме,
чтобы максимально соответствовать должности {position}. Соискатель написал резюме, которое содержит навыки, образование соискателя,
его опыт работы. Так же приложена вакансия. Проведи анализ относительно данной вакансии, напиши рекомендации по корректировке резюме,
чтобы максимально соответствовать должности {position} и максимально подходить к данном вакансии'''

resume = gpt.get_resume()
user_content = f'Резюме соискателя:\n{resume}\n\nВакансия:\n{vacancy2}'
gpt.gpt_gen_print(system_content, user_content)

Анализ вакансии и рекомендации по корректировке резюме:

1. Образование:
В резюме указано два высших образования, но они не связаны напрямую с областью Data Science. Рекомендуется выделить в резюме только наиболее релевантное образование, а именно:
- Университет Синергия (2022 - настоящее время) - Data Scientist
- НИУ ИТМО, Санкт-Петербург (2021 - настоящее время) - Технологии анализа данных

2. Ключевые навыки:
В резюме указаны некоторые технические навыки, но их список можно расширить, чтобы максимально соответствовать требованиям вакансии. Рекомендуется добавить следующие навыки:
- SQL
- Machine Learning
- Cloud platforms (Azure ML, Google Cloud Platform ML, AWS SageMaker, Yandex Cloud Platform)
- Data visualization (Power BI, ggplot, plotly)

3. Опыт работы:
В резюме указан опыт работы, но не все проекты и задачи связаны с Data Science. Рекомендуется выделить только те проекты и задачи, которые относятся к Data Science и максимально соответствуют требованиям вакансии. Например:
- П

In [ ]:
system_content = f'''Ты опытный hr консультант, который консультирует соискателя на должность {position}.
Соискатель написал резюме, так же приложена вакансия. Проведи анализ резюме соискателя относительно данной вакансии.
Напиши в процентах, насколько данный кандидат соответствует данной вакансии, а так же перечисли то, чего ему не хватате,
чтобы устроиться на данную вакансию'''

resume = gpt.get_resume()
user_content = f'Резюме соискателя:\n{resume}\n\nВакансия:\n{vacancy2}'
gpt.gpt_gen_print(system_content, user_content, quiet=True)

Анализ резюме соискателя относительно вакансии Data Scientist:

1. Образование и опыт:
   - Соискатель имеет образование в области экономики, что не является прямым соответствием требованиям вакансии, где ожидается образование в области статистики, математики или информатики. (-10%)

2. Навыки и опыт работы:
   - Соискатель обладает навыками работы с Python, Numpy, Matplotlib, Excel, Power BI, что соответствует требованиям вакансии. (+20%)
   - Опыт работы в области Data Science отсутствует. (-10%)
   - Опыт работы в аналитической сфере присутствует, но не связан с Data Science. (+5%)

3. Личные качества:
   - Соискатель обладает дисциплинированностью, стремлением к профессиональному и карьерному росту, умением работать с большим объемом информации, хранить конфиденциальную информацию. Эти качества соответствуют требованиям вакансии. (+10%)
   - Однако, отсутствует информация о коммуникационных навыках и способности решать проблемы, что является важным для работы Data Scientist. (-5%)


### Сопроводительное письмо

In [ ]:
promt = f'''Ты опытный hr консультант, который помогает соискателю грамотно написать сопроводительное письмо.
Напиши сопроводительное письмо на должность "{position}" от лица соискателя на основании его резюме и образца сопроводительного письма.
Выдели навыки, опыт и личные качества соискателя, связанные с работой, которые делают его подходящим для этой должности.
В образце предоставлен текст сопроводительного письма, а в квадратных скобках указано то, что нужно подставить в текст исходя из резюме соискателя'''

In [ ]:
sample = '''
Уважаемый [Имя получателя],

Я пишу, чтобы подать заявку на должность старшего специалиста по данным в Сан-Франциско / удаленно в [название компании].

У меня [квалификация, пример: информатика, наука о данных] из [университет / институт, пример: МФТИ 2017], [местонахождение университета] и [количество лет] лет опыта работы с машинным обучением, глубоким обучением, RL, GAN, компьютерное зрение, распознавание лиц и т. Д. В таких компаниях, как Google, Яндекс, Huawei [названия компаний]

Мои основные результаты как специалист по анализу данных: [назовите 3-4 основных результата как ML, DL эксперт, используя ключевые слова]. На моей предыдущей должности в [Назовите компанию: Samsung, Яндекс, Google. Лучше — известная компания], я отвечал за [обсудить ваши основные обязанности]

Согласно требованиям, предъявляемым к должности в [компании], мой технический стек — это Python, Pytorch, Tensorflow, Machine Learning, Deep Learning, Reinforcement Learning, Graph methods.

В моем резюме вы заметите, что я [важно описать опыт, технические навыки, компетенции, которые непосредственно отвечают требованиям, перечисленным в описании вакансии специалиста по данным].

Я обращаюсь в [компанию], потому что восхищаюсь вашими [назовите 2 или 3 факта о компании и ее проектах, связанных с ML, DL в приоритетах, которые вам действительно нравятся в ней]. Как специалист по Data Science, я был бы рад присоединиться к команде профессионалов и использовать свой опыт в алгоритмах DS, ML, RL для получения результатов [компании].

Спасибо, что нашли время рассмотреть мою заявку. С нетерпением жду Вашего ответа.

С уважением,

[Ваше имя, номер телефона]
'''

In [ ]:
resume = gpt.get_resume()
system_content = promt
user_content = resume
assistant_content = sample
gpt.gpt_gen_print(system_content, user_content, assistant_content)

Уважаемый [Имя],

Я хочу выразить свой интерес к вакансии Data Scientist в вашей компании. У меня есть опыт работы в области аналитики и разработки данных, а также навыки, необходимые для успешного выполнения этой роли.

Я получил высшее образование в СПбГЭУ и Санкт-Петербургском государственном инженерно-экономическом университете "ИНЖЭКОН" по специальности экономика. Кроме того, я прохожу курсы по Data Science в Университете Синергия и НИУ ИТМО, чтобы углубить свои знания и навыки в этой области.

У меня есть опыт работы в качестве Data Scientist на фрилансе, где я занимался разработкой отчетов в Power BI и созданием моделей прогнозирования прибыли с использованием линейной регрессии. Я также работал с большим объемом данных и умею эффективно анализировать их с помощью Python, Numpy и Matplotlib.

В моей последней работе в компании Элерон я занимался аналитикой в Power BI и Excel, использовал базы данных и создавал автоматизированные отчеты. Я также имею опыт работы с 1C и SQL для сб

In [ ]:
# Добавляем "о компании"

In [ ]:
promt = f'''Ты опытный hr консультант, который помогает соискателю грамотно написать сопроводительное письмо.
Напиши сопроводительное письмо на должность "{position}" от лица соискателя на основании его резюме и образца сопроводительного письма.
Выдели навыки, опыт и личные качества соискателя, связанные с работой, которые делают его подходящим для этой должности.
Предоставлен текст о компании и образец сопроводительного письма.
В образце сопроводительного письма в квадратных скобках указано то, что нужно подставить в текст исходя из резюме соискателя.'''

In [ ]:
vacancy1_about = '''
Многофункциональный общий центр обслуживания (Филиал ОЦО) — это крупнейший среди западных и российских корпораций центр по обслуживанию внутренних клиентов (сотрудников «Ростелекома»). Филиал ОЦО был создан в 2012 году по опыту лучших мировых практик. Наши девять центров расположены в 6 городах: Нижнем Новгороде, Екатеринбурге, Новосибирске, Москве, Воронеже, Ярославле. Более 4000 сотрудников, знающих свое дело и болеющих за него всей душой, обслуживают всю территорию представленности «Ростелекома». Мы помогаем компании сосредоточиться на ее стратегическом бизнесе, переняв вспомогательные функции на себя.
'''

In [ ]:
resume = gpt.get_resume()
system_content = promt
user_content = resume
assistant_content = f'Текст о компании:\n{vacancy1_about}\nОбразец сопроводительного письма:\n{sample}'
gpt.gpt_gen_print(system_content, user_content, assistant_content)

Уважаемый [Имя рекрутера],

Я хочу выразить свой интерес к вакансии Data Scientist в вашей компании. Уверен, что мой опыт и навыки позволят мне успешно выполнять задачи, связанные с анализом данных и разработкой моделей.

Я обладаю широким спектром навыков, необходимых для работы Data Scientist. Мой опыт включает использование языка программирования Python, библиотеки Numpy и инструментов визуализации данных, таких как Matplotlib. Я также имею опыт работы с Excel и Power BI, что позволяет мне эффективно анализировать и представлять данные.

Моя образовательная подготовка в области экономики и анализа данных дает мне прочные теоретические знания, которые я успешно применяю на практике. Я также постоянно обучаюсь новым технологиям и методам анализа данных, чтобы быть в курсе последних тенденций в этой области.

Я дисциплинирован и всегда соблюдаю сроки выполнения задач. Мне нравится работать с большим объемом информации и я умею хранить конфиденциальную информацию. Я также обладаю способ

In [ ]:
gpt.gpt_gen_print(system_content, user_content, assistant_content, temperature=0.2)

Уважаемый [Имя рекрутера],

Я хочу выразить свой интерес к вакансии Data Scientist в вашей компании. Уверен, что мой опыт и навыки позволят мне успешно выполнять задачи, связанные с анализом данных и разработкой моделей.

Я обладаю широким спектром навыков, необходимых для работы Data Scientist. Мой опыт включает использование языка программирования Python, библиотеки Numpy и инструментов визуализации данных, таких как Matplotlib. Я также имею опыт работы с Excel и Power BI, что позволяет мне эффективно анализировать и представлять данные.

Моя образовательная подготовка в области экономики и анализа данных дает мне теоретическую базу для работы Data Scientist. Я изучал экономику и мировую экономику в СПбГЭУ и Санкт-Петербургском государственном инженерно-экономическом университете "ИНЖЭКОН". Кроме того, я прошел курсы по технологиям анализа данных в НИУ ИТМО.

Я также активно развиваю свои навыки и знания, посещая курсы по деловому английскому и экономике и финансам в БФУ им. И. Канта

# Эксперименты 4

## Резюме

In [ ]:
resume1 = {
'name': 'Иван Петрович Сидоров',
'city': 'Новополис',
'position': 'Data Scientist',

'contacts': [
    {'email': 'my@email.com'},
    {'Телефон': '+7(999)999-99-99'},
],

'about': '''
Сдан экзамен P1 CIMA
Средний бал: 4,6

Имея изначально склонность к аналитике, решил добавить технологические стек и уже более углублено ее развивать в направлении Data Science.

Личные качества:
- Нахожу общий язык с коллегами
- Дисциплинирован.
- Постоянно обучаюсь новому.
- Стремлюсь к профессиональному и карьерному росту.
- В работе соблюдаю сроки выполнения задач.
- Способен длительное время заниматься решением одной проблемы.
- Умею работать с большим объёмом информации.
- Умею хранить конфиденциальную информацию.
- Способен исследовать, проверять факты и собирать точные данные.
''',

'skills': [
    'Python',
    'Numpy',
    'Matplotlib',
    'Excel',
    'Power BI',
    '1C',
    ],

'experience': [
    {
        'date_start': '06.2022',
        'date_end': '',
        'organisation': 'Элерон (Европейская химчистка Apetta)',
        'city': 'Санкт-Петербург',
        'web': 'apetta.ru',
        'industry': 'Услуги для населения',
        'position': 'Бизнес-аналитик',
        'experience_type': 'Основная работа',
        'activity': 'Аналитика в Power BI, Excel. Использование базы данных. Построение автоматизированных отчетов.',
    },
    {
        'date_start': '09.2021',
        'date_end': '',
        'organisation': 'Проектная деятельность',
        'city': 'Санкт-Петербург',
        'web': 'github.com/Litvinov-Anton/Portfolio',
        'industry': '',
        'position': 'Data Scientist',
        'experience_type': 'Фриланс',
        'activity': '''
Деятельность:
Коммерческий проект по созданию отчета в Power BI, для компании Data Sfera.

Личный проект. Создал проект по отнесению кондитерских изделий к классу с помощью логистической регрессии, на основании данных Kaggle.

Личный проект. Создал несколько финансовых отчетов в Power BI, DataStudio.

Личный проект. Создал проекты по предсказанию прибыли с помощью линейной регрессии, на основании данных Kaggle.

Личный проект. Составил финансовую модель с помощью Excel.

Личный проект. Создал проект по классификации с помощью метода К-Средних, на основании данных Kaggle.
        ''',
    },
    {
        'date_start': '10.2021',
        'date_end': '11.2021',
        'organisation': 'ООО "УК "Клин-Тим" (промышленный клининг)',
        'city': 'Санкт-Петербург',
        'web': 'cleanteam.ru',
        'industry': '',
        'position': 'Экономист',
        'experience_type': 'Основная работа',
        'activity': '''
- Ведение управленческого учета компании (2 ЦФО)
- Сбор данных и формирование отчетов по маржинальности объектов (1С УПП, Excel)
- План-факторный анализ полученных данных, построение отчетности и визуализация данных (Power BI)
- Ввод данных смет в программу учета
- Подготовка аналитических справок, статистик по итогам работы за период.
- Ведение электронного архива смет

Ключевые результат:
внедрил инструмента PowerBI визуализации в процесс предоставления отчётности
''',
    },
    {
        'date_start': '06.2020',
        'date_end': '05.2021',
        'organisation': '"Терра -Авто" КЦ" (ГК «Прагматика») - один из крупнейших Российских автодилеров',
        'city': 'Санкт-Петербург',
        'web': 'www.terravto.ru',
        'industry': 'Автомобильный бизнес',
        'position': 'Экономист',
        'experience_type': 'Основная работа',
        'activity': '''
-Составление бюджета компании (2 ЦФО): планирование показателей операционного производства, выручки от продаж, ФОТа, (OPEX, CAPEX)
-План-факторный анализ полученных данных
-Составление управленческой отчетности (Excel, SQL, 1С)
-Составление экономической и финансовой отчётности для внешних контрагентов по запросу
- Контроль исполнения платежного календаря
Ключевые результаты:
Разработал и внедрил систему для расчета мотивации персонала по компании за счет введения углубленной аналитики по продажам.
''',
    },
],

'education': [
    {
        'date_start': '2017',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'СПбГЭУ',
        'faculty': 'Экономики, Мировая экономика',
    },
    {
        'date_start': '2015',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'Санкт-Петербургский государственный инженерно-экономический университет "ИНЖЭКОН", Санкт-Петербург',
        'faculty': 'Экономики, Национальная экономика',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Техническая Академия РосАтома',
        'faculty': 'Деловой английский',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Экономика и финансы',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Разработка и реализация маркетинговой стратегии',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'БФУ им. И. Канта',
        'faculty': 'Бизнес-модели компании и операционная эффективность',
    },
    {
        'date_start': '2022',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Университет Синергия',
        'faculty': 'Data Scientist',
    },
    {
        'date_start': '2021',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'НИУ ИТМО, Санкт-Петербург',
        'faculty': 'Технологии анализа данных',
    },
]
}

In [ ]:
resume2 = {
'name': 'Петр Сидорович Иванов',
'city': 'Дальнегорск',
'position': 'Data Scientist',

'contacts': [
    {'email': 'my2@email.com'},
    {'Телефон': '+7(999)999-99-90'},
],

'about': '''
Выполненные работы: "Прогнозирование оттока клиентов на предприятии".
Средний балл: 4.9
Технические навыки:
- Apache Spark
- Hadoop
- Python(Pandas, Numpy, Matplotlib,Sklearn xgboost, lightgbm, catboost, pytorch, hyperopt, optuna)
- Oracle PL/SQL
- Wolfram Mathematica
Сфера профессиональных интересов:
- Базы данных
- Математические методы в экономике
- Статистический анализ
- Эконометрическое моделирование
- Машинное обучение
- Нейронные сети
Профессиональные навыки:
- Базы данных
- Машинное обучение
- Нейронные сети
- Эконометрическое моделирование
- Анализ временных рядов
- Анализ данных
- Английский язык
- Бухгалтерский учет
''',

'skills': [
    'Python',
    'Apache Spark',
    'Sklearn',
    'LightGBM',
    'CatBoost',
    'hyperopt',
    'Pytorch',
    'Linux',
    'Matplotlib',
    'Docker',
    'Kafka',
    'Nats',
    'Machine Learning',
    'Git',
    'SQL',
    'FastApi',
    'MlFlow',
    'Комбинаторная оптимизация',
    'Kubernetes',
    'Oracle Pl/SQL',
    'Big Data',
],

'experience': [
    {
        'date_start': '10.2021',
        'date_end': '',
        'organisation': 'KAMAZ DIGITAL',
        'city': 'Санкт-Петербург',
        'web': '',
        'industry': '',
        'position': 'Data Scientist',
        'experience_type': 'Основная работа',
        'activity': '''
1) Разработка и внедрение классических моделей машинного обучения (классификация, регрессия):
1.1) Предсказание стоимости перевозки груза
2) Разработка моделей комбинаторной оптимизации (vrp, с помощью ortools python):
2.1) Построение маршрутов для автопарка грузовых автомобилей
2.2) Построение маршрутов пассажирских перевозок (Сервис "По пути" Мосгортранса)
3) Работа с данными телеметрии / обработка сигналов:
3.1) Оценка качества вождение транспортного средства
3.2) Определение точек заправки транспортного средства
4) Внедрение распределённых программных брокеров сообщений kafka / Nats
5) FastApi, asincio, docker, mlflow
6) Работа с базами данных: postgresql, clickhouse
''',
    },
    {
        'date_start': '09.2020',
        'date_end': '09.2021',
        'organisation': 'Сбер',
        'city': 'Санкт-Петербург',
        'web': '',
        'industry': 'Финансовый сектор',
        'position': 'Data Scientist',
        'experience_type': 'Основная работа',
        'activity': '''
1) Полный цикл разработки моделей машинного обучения:
1.1) Классификации деятельности организаций на основе их транзакционной активности
1.2) Прогнозирование количества оборотов активов у Юр. лиц
2) Создание витрин данных
3) Анализ временных рядов, классификация, кластеризация на больших данных
4) Работа с большими данными на кластере Hadoop с использованием фреймворка Spark
5) Построение ETL процессов в SQL Oracle.
''',
    },
],

'education': [
    {
        'date_start': '2021',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'Санкт-Петербургский государственный экономический университет, Санкт-Петербург',
        'faculty': 'Экономики, Национальная экономика',
    },
    {
        'date_start': '2021',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'NEWPROLAB',
        'faculty': 'NEW PROFESSIONS LAB, Apache Spark для задач анализа данных',
    },
    {
        'date_start': '2020',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Coursera',
        'faculty': 'МФТИ, Яндекс, Математика и python для анализа данных',
    },
    {
        'date_start': '2020',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Coursera',
        'faculty': 'МФТИ, Яндекс, Обучение на размещенных данных',
    },
    {
        'date_start': '2020',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Coursera',
        'faculty': 'МФТИ, Яндекс, Поиск структуры в данных',
    },
]
}

In [ ]:
resume3 = {
'name': 'Сидор Иванович Петров',
'city': 'Неогорск',
'position': 'Data Scientist',

'contacts': [
    {'email': 'my3@email.com'},
    {'Телефон': '+7(999)999-99-00'},
],

'about': '''
''',

'skills': [
    'Python',
    'Data Analysis',
    'Data Mining',
    'Keras',
    'Pytorch',
    'Torch',
    'CudaNN',
    'Docker',
    'TorchRT',
    'Nats',
    'MLFlow',
    'Git',
    'SQL',
    'OpenCV',
    'DVC',
    'Github actions',
    'Kubernetes',
    'Big Data',
    'NLP',
    'LLM',
],

'experience': [
    {
        'date_start': '02.2022',
        'date_end': '',
        'organisation': 'ANNA Money',
        'city': 'Великобритания',
        'web': 'anna.money',
        'industry': 'Финансовый сектор',
        'position': 'Data Scientist',
        'experience_type': 'Основная работа',
        'activity': '''
Создание сервиса в многосервисной инфраструктуре, обслуживание сервиса, создание модели категоризации транзакций,
проверка гипотез по улучшению модели, создание дашбордов с бизнес метриками
''',
    },
    {
        'date_start': '08.2019',
        'date_end': '01.2022',
        'organisation': 'Газпромнефть НТЦ',
        'city': 'Санкт-Петербург',
        'web': 'www.gazprom-neft.ru',
        'industry': 'Информационные технологии',
        'position': 'Руководитель направления центра компетенций по когнитивным технологиям',
        'experience_type': 'Основная работа',
        'activity': '''
Поддержка прод ML решений, разработка ML решений на основе CNN для решения задач геологоразведки
(детекция + сегментация, генерация синтетики), разворачивание, доработка и поддержка систем управления ML экспериментами,
оптимизация инференса, конвертация моделей между фреймворками
''',
    },
    {
        'date_start': '01.2017',
        'date_end': '08.2019',
        'organisation': 'Газпромнефть НТЦ',
        'city': 'Санкт-Петербург',
        'web': '',
        'industry': 'Информационные технологии',
        'position': 'Главный специалист',
        'experience_type': 'Основная работа',
        'activity': '''
Разработка прикладного программного обеспечения
Исследовательская и научная работа
Разработка графических интерфейсов
Сопровождение проектов и экспертиза
Разработка моделей машинного обучения
Анализ промысловых данных
''',
    },
    {
        'date_start': '01.2018',
        'date_end': '01.2019',
        'organisation': 'Научно-образовательный центр "Газпромнефть-Политех"',
        'city': 'Санкт-Петербург',
        'web': '',
        'industry': 'Информационные технологии',
        'position': 'Ведущий специалист',
        'experience_type': 'Основная работа',
        'activity': '''
Разработка прикладного программного обеспечения
Исследовательская и научная работа
Разработка графических интерфейсов
Сопровождение проектов и экспертиза
''',
    },
],

'education': [
    {
        'date_start': '2018',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'Санкт-Петербургский государственный политехнический университет, Санкт-Петербург',
        'faculty': 'Институт прикладной математики и механики, Математическое моделирование процессов нефтегазодобычи',
    },
    {
        'date_start': '2018',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'Санкт-Петербургский государственный политехнический университет, Санкт-Петербург',
        'faculty': 'Прикладной математики и механики, Математическое моделирование процессов газонефтедобычи',
    },
    {
        'date_start': '2018',
        'date_end': '',
        'education_type': 'Высшее образование',
        'institution': 'Санкт-Петербургский государственный политехнический университет, Санкт-Петербург',
        'faculty': 'Факультет технической кибернетики, Системный анализ и управление',
    },
    {
        'date_start': '2020',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Convolutional Neural Networks',
        'faculty': 'Stanford EDU, CNN for visual recognition',
    },
    {
        'date_start': '2019',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Mlcourse.ai',
        'faculty': 'Ods, ML course fall 2019',
    },
    {
        'date_start': '2019',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Machine Learning and Data Analysis',
        'faculty': 'Yandex',
    },
    {
        'date_start': '2019',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Machine learning summer school',
        'faculty': 'Skoltech',
    },
    {
        'date_start': '2019',
        'date_end': '',
        'education_type': 'Курсы',
        'institution': 'Обучение на размеченных данных',
        'faculty': 'COURSERA, Data science',
    },
]
}

## Вакансии

In [ ]:
vacancy1 = {
    'about': '''
Многофункциональный общий центр обслуживания (Филиал ОЦО) — это крупнейший среди западных и российских корпораций центр по обслуживанию внутренних клиентов
(сотрудников «Ростелекома»). Филиал ОЦО был создан в 2012 году по опыту лучших мировых практик. Наши девять центров расположены в 6 городах:
Нижнем Новгороде, Екатеринбурге, Новосибирске, Москве, Воронеже, Ярославле. Более 4000 сотрудников, знающих свое дело и болеющих за него всей душой,
обслуживают всю территорию представленности «Ростелекома». Мы помогаем компании сосредоточиться на ее стратегическом бизнесе, переняв вспомогательные функции на себя.
''',

    'requirements': '''
Задачи
- Разработка скоринговых моделей оценки контрагентов физических и юридических лиц (сегменты B2C, B2B, B2G)
- Разработка моделей оптимизации стратегий сбора задолженности (collection scoring)
- Разработка ML моделей (включая нейросети) для автоматизации и повышения эффективности операционных процессов
- Разработка ML моделей по анализу и настройке голосовых роботов
- Участие во внедрении и тестировании ML моделей, ПО и инфраструктуры для ML
- Мониторинг эффективности действующих моделей
- Анализ эффективности внутренних и внешних информационных источников

Нам важно
- Опыт разработки скоринговых/ML моделей
- Навык работы с БД
- Владение SQL и Python
- Знание подходов к оценке эффективности моделей и финансовой эффективности их внедрения
- Навыки построения аналитических отчетов и умение визуализировать данные
- Хорошую аналитическую и математическую подготовку
- Умение копаться в данных, генерировать гипотезы, находить инсайты
- Будет большим плюсом опыт менторинга других аналитиков
''',

    'skills': [],
}

In [ ]:
vacancy2 = {
    'about': '''
''',

    'requirements': '''
Вы будете:

    Взаимодействовать с заинтересованными сторонами в рамках всей организации, чтобы определить осуществимость использования данных компании для формирования бизнес-решений и монетизации данных
    Извлекать и анализировать данные из баз данных компаний для оптимизации инструментов, процессов
    Производить пользовательские модели данных и алгоритмов для применения к наборам данных;
    Формулировать процессы и инструменты для мониторинга и анализа производительности модели и точности данных;
    Представлять результаты моделирования для бизнес-аудитории, включая руководителей организации;
    Реализовывать стратегию управления бизнес-данными и давать рекомендации
    Проектировать, рефакторить, разворачивать дата продукты;
    Выдвигать в организации использование подготовленных внутри дата продуктов с помощью презентаций, документации и совместной работы бизнес функций;
    Подготавливать инструменты и автоматизировать рабочие процессы;
    Превращать неструктурированные недостатки и болевые точки в конкретные бизнес и технические требования для проверки гипотез или производства дата продуктов.

Мы ожидаем:

    Образование и соответствующий опыт в области статистики, математики, информатики или смежных областях;
    Уверенное знание SQL, Python, алгоритмов машинного обучения и статистики;
    Подтвержденный опыт создания пайплайнов машинного обучения, работающих в продуктивной среде разработки (обучение, оценка, использование в системах реального времени);
    Опыт использования облачных платформ (Azure ML, Google Cloud Platform ML, AWS SageMaker, Yandex Cloud Platform) и развертывания аналитики в продуктивной среде;
    Опыт работы со стеком Azure (Data Lake, Database, Data Factory, Data Brick и т. д.) и/или Yandex Cloud (Data Proc, Data Sphere и т. д.) – будет плюсом;
    Опыт визуализации данных с использованием Power BI/ggplot/plotly/ – будет плюсом;
    Высокие навыки коммуникации, решения проблем, стремление к обучению;
    Самостоятельного специалиста, с высокой ответственностью, готового привносить структуру в неструктурированные ситуации;
    Средний или выше среднего уровень знания английского языка (разговорный и письменный).
''',

    'skills': [],
}

In [ ]:
vacancy3 = {
    'about': '''
AnyQuery — SaaS сервис AI поиска для eCommerce, обрабатывает миллиарды поисковых запросов на 450 интернет-магазинах (Самокат, Золотое Яблоков, ЦУМ, Эльдорадо, Петрович, Азбука Вкуса и тд)
Вдобавок к AnyQuery, у нас есть два новых продукта AnyRecs — SaaS сервис персонализации и товарных рекомендаций, AnyCollections — SaaS сервис для автоматического создания посадочных страниц интернет-магазинов
У нас огромная экспертиза в машинном обучении, high load системах.
В текущей команде около 90 человек, компания продолжает расти и ищет Data Scientist.
''',

    'requirements': '''
Ключевые задачи:
- разработать новую векторную модель товаров и интересов пользователя на основе 30+ млрд пользовательских событий и 60 млн товаров
- ускорить проверки гипотез в команде ранжирования - разработать инструменты для работы с датасетами и пайпланы создания фичей.

Что надо делать:
- Разрабатывать новые модели векторного поиска и вектров
- Разработать алгоритмы товарных рекомендаций похожих товаров на векторных моделях
- С командой аналитиков ускорить процесс создания дата сетов для обучения моделей и создания фичей

Что нужно уметь :
- Хорошо разбираться в векторных моделях и NLP (DSSM, LLM, FastText)
- Уметь писать готовый к production код (Python)
- Иметь математический бекграунд (профильное техническое/математическое образование, хорошее понимание мат.статистики, линейной алгебры)
- Опыт работы с текстовыми данными или задачами поиска будет серьезным плюсом
- Java/Scala/Elasticsearch/C++ будет плюсом
''',

    'skills': [
        'SQL',
        'Python',
        'Big Data',
        'Tensorflow',
        'Pytorch',
        'SciPy',
        'Numpy',
        'Pandas',
        'NLP',
        'LLM',
    ]
}

## Соответствие ключевых навыков вакансиям

### Через алгоритмы

In [ ]:
def find_top_matching_resumes(resume_list, job_skills):
    # создаем список словарей, где каждый словарь содержит информацию о резюме и процент соответствия skills
    matching_scores = []
    for resume in resume_list:
        resume_skills = resume['skills']
        common_skills = set(resume_skills) & set(job_skills)
        matching_score = len(common_skills) / len(job_skills) * 100
        matching_scores.append({'resume': resume, 'matching_score': matching_score})

    # сортируем по убыванию процента соответствия и выбираем топ-10
    # top_matching_resumes = sorted(matching_scores, key=lambda x: x['matching_score'], reverse=True)[:10]
    top_matching_resumes = sorted(matching_scores, key=lambda x: x['matching_score'], reverse=True)

    return top_matching_resumes

In [ ]:
resume_list = [resume1, resume2, resume3]
job_skills = vacancy3['skills']

In [ ]:
top_matching_resumes = find_top_matching_resumes(resume_list, job_skills)
for item in top_matching_resumes:
    print(item['resume']['name'], item['matching_score'])

Сидор Иванович Петров 60.0
Петр Сидорович Иванов 40.0
Иван Петрович Сидоров 20.0


Данные результаты показывают совпадение в процентах между требованиями в вакансии и наличия ключевых навыков в резюме

### Через Embedding

In [ ]:
document = []
for resume in resume_list:
    page_content = ', '.join(resume['skills'])
    metadata = {'source': resume['name']}
    doc = Document(page_content=page_content, metadata=metadata)
    document.append(doc)

In [ ]:
db = FAISS.from_documents(document, OpenAIEmbeddings())
query = ', '.join(vacancy3['skills'])
results = db.similarity_search(query, k=5)

In [ ]:
results

[Document(page_content='Python, Data Analysis, Data Mining, Keras, Pytorch, Torch, CudaNN, Docker, TorchRT, Nats, MLFlow, Git, SQL, OpenCV, DVC, Github actions, Kubernetes, Big Data, NLP, LLM', metadata={'source': 'Сидор Иванович Петров'}),
 Document(page_content='Python, Apache Spark, Sklearn, LightGBM, CatBoost, hyperopt, Pytorch, Linux, Matplotlib, Docker, Kafka, Nats, Machine Learning, Git, SQL, FastApi, MlFlow, Комбинаторная оптимизация, Kubernetes, Oracle Pl/SQL, Big Data', metadata={'source': 'Петр Сидорович Иванов'}),
 Document(page_content='Python, Numpy, Matplotlib, Excel, Power BI, 1C', metadata={'source': 'Иван Петрович Сидоров'})]

In [ ]:
for result in results:
    print(result.metadata['source'])

Сидор Иванович Петров
Петр Сидорович Иванов
Иван Петрович Сидоров


In [ ]:
results = db.similarity_search_with_score(query, k=5)

In [ ]:
for result, score in results:
    print(result.metadata['source'], score)

Сидор Иванович Петров 0.14874074
Петр Сидорович Иванов 0.22554828
Иван Петрович Сидоров 0.2599393


Данные результаты показывают наиболее близкие резюме по ключевым навыкам к требованиям в вакансии

## Соответствие резюме вакансиям (Embedding)

Поиск через embedding. Минусы библиотеки LangChain:
- нужен ключ OpenAI
- ограничения по запускам в единицу времени (в ограниченной версии ключа API)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.schema.document import Document

### По одной единице опыта работы

*Вектором является вся вакансия целиком (без описания компании). Он сопоставляется с вектором, который является опытом работы соискателя в каждой отдельной компании*

In [ ]:
document = []
for resume in resume_list:
    for experience in resume['experience']:
        page_content = experience['activity']
        metadata = {'source': resume['name']}
        doc = Document(page_content=page_content, metadata=metadata)
        document.append(doc)

**Вакансия 1**

In [ ]:
db = FAISS.from_documents(document, OpenAIEmbeddings())
query = vacancy1['requirements']
results = db.similarity_search(query, k=5)

In [ ]:
results

[Document(page_content='\n1) Разработка и внедрение классических моделей машинного обучения (классификация, регрессия):\n1.1) Предсказание стоимости перевозки груза\n2) Разработка моделей комбинаторной оптимизации (vrp, с помощью ortools python):\n2.1) Построение маршрутов для автопарка грузовых автомобилей\n2.2) Построение маршрутов пассажирских перевозок (Сервис "По пути" Мосгортранса)\n3) Работа с данными телеметрии / обработка сигналов:\n3.1) Оценка качества вождение транспортного средства\n3.2) Определение точек заправки транспортного средства\n4) Внедрение распределённых программных брокеров сообщений kafka / Nats\n5) FastApi, asincio, docker, mlflow\n6) Работа с базами данных: postgresql, clickhouse\n', metadata={'source': 'Петр Сидорович Иванов'}),
 Document(page_content='\n1) Полный цикл разработки моделей машинного обучения:\n1.1) Классификации деятельности организаций на основе их транзакционной активности\n1.2) Прогнозирование количества оборотов активов у Юр. лиц\n2) Созда

In [ ]:
for result in results:
    print(result.metadata['source'])

Петр Сидорович Иванов
Петр Сидорович Иванов
Сидор Иванович Петров
Сидор Иванович Петров
Иван Петрович Сидоров


In [ ]:
results = db.similarity_search_with_score(query, k=5)

In [ ]:
results

[(Document(page_content='\n1) Разработка и внедрение классических моделей машинного обучения (классификация, регрессия):\n1.1) Предсказание стоимости перевозки груза\n2) Разработка моделей комбинаторной оптимизации (vrp, с помощью ortools python):\n2.1) Построение маршрутов для автопарка грузовых автомобилей\n2.2) Построение маршрутов пассажирских перевозок (Сервис "По пути" Мосгортранса)\n3) Работа с данными телеметрии / обработка сигналов:\n3.1) Оценка качества вождение транспортного средства\n3.2) Определение точек заправки транспортного средства\n4) Внедрение распределённых программных брокеров сообщений kafka / Nats\n5) FastApi, asincio, docker, mlflow\n6) Работа с базами данных: postgresql, clickhouse\n', metadata={'source': 'Петр Сидорович Иванов'}),
  0.21025267),
 (Document(page_content='\n1) Полный цикл разработки моделей машинного обучения:\n1.1) Классификации деятельности организаций на основе их транзакционной активности\n1.2) Прогнозирование количества оборотов активов у 

In [ ]:
for result, score in results:
    print(result.metadata['source'], score)

Петр Сидорович Иванов 0.21025267
Петр Сидорович Иванов 0.23656939
Сидор Иванович Петров 0.24710242
Сидор Иванович Петров 0.27249053
Иван Петрович Сидоров 0.2785055


**Вакансия 2**

In [ ]:
db = FAISS.from_documents(document, OpenAIEmbeddings())
query = vacancy2['requirements']
results = db.similarity_search_with_score(query, k=5)

In [ ]:
for result, score in results:
    print(result.metadata['source'], score)

Петр Сидорович Иванов 0.25636396
Сидор Иванович Петров 0.2755815
Петр Сидорович Иванов 0.27788424
Иван Петрович Сидоров 0.28717196
Иван Петрович Сидоров 0.3105334


**Вакансия 3**

In [ ]:
db = FAISS.from_documents(document, OpenAIEmbeddings())
query = vacancy3['requirements']
results = db.similarity_search_with_score(query, k=5)

In [ ]:
for result, score in results:
    print(result.metadata['source'], score)

Петр Сидорович Иванов 0.2735328
Петр Сидорович Иванов 0.2752887
Сидор Иванович Петров 0.31536686
Сидор Иванович Петров 0.32624656
Сидор Иванович Петров 0.3416112


*Далее, в качестве вектора используется не целая вакансия, а только требования. Сопоставление так же идёт с вектором, образованным из опыта работы в одной компании*

**Вакансия 3**

In [ ]:
query = '''
- Хорошо разбираться в векторных моделях и NLP (DSSM, LLM, FastText)
- Уметь писать готовый к production код (Python)
- Иметь математический бекграунд (профильное техническое/математическое образование, хорошее понимание мат.статистики, линейной алгебры)
- Опыт работы с текстовыми данными или задачами поиска будет серьезным плюсом
- Java/Scala/Elasticsearch/C++ будет плюсом
'''

In [ ]:
db = FAISS.from_documents(document, OpenAIEmbeddings())
results = db.similarity_search_with_score(query, k=5)

In [ ]:
for result, score in results:
    print(result.metadata['source'], score)

Петр Сидорович Иванов 0.27197373
Сидор Иванович Петров 0.306318
Петр Сидорович Иванов 0.3094887
Сидор Иванович Петров 0.34212795
Сидор Иванович Петров 0.34294838


### По всему опыту работы

*Здесь, вектор из вакансии целиком (без описания компании) сопоставляется с вектором всего опыта работы соискателя (по всему резюме)*

**Вакансия 3**

In [ ]:
document = []
for resume in resume_list:
    activity = []
    for experience in resume['experience']:
        activity.append(experience['activity'])
    page_content = '\n\n'.join(activity)
    metadata = {'source': resume['name']}
    doc = Document(page_content=page_content, metadata=metadata)
    document.append(doc)

In [ ]:
db = FAISS.from_documents(document, OpenAIEmbeddings())
query = vacancy3['requirements']
results = db.similarity_search_with_score(query, k=5)

In [ ]:
for result, score in results:
    print(result.metadata['source'], score)

Петр Сидорович Иванов 0.2615984
Сидор Иванович Петров 0.2854032
Иван Петрович Сидоров 0.34869385


## Выводы

- LangChain хорошо себя показывает не только для нахождения чанков, релевантных запросу, а ещё и для поиска соответствия ключевых навыков соискателя ключевым навыкам вакансии
- Есть однозначная корреляция между алгоритмическим поиском соответствия ключевых навыков резюме и поиску через embedding
- LangChain так же показывает результаты соответствия требований вакансии и данных в резюме. С его помощью можно найти более релевантные резюме по опыту работы
- Для более точных результатов необходимо проводить дополнительные исследования и эксперименты относительно того, какие части резюме и вакансий необходимо выделять для преобразования в векторы
- **В целом, эксперименты с embedding показали результат.** В веб-сервисе можно сделать функцию автоматического преборазования резюме и вакансий в embedding, что позволит мговенно их сопоставлять и выявлять наилучшие совпадения